# Predicting mortality in the MIMIC III data

# Introduction

The goal of this project is to apply a variety of machine learning models to the problem of predicting mortality from Electronic Health Record (EHR) data (both structured and unstructured). More specifically, our dataset consists of hourly observations of 104 clinically-aggregated covariates related to vital signs and lab results, static features (age, gender, and ethnicity), as well as free-form patient chart notes from the MIMIC III Database <cite data-cite="11046887/SG9TBKZH"></cite>. We use this data to predict mortality (6+ hours out) for ICU patients based on the first 12, 24, and 48 hours of their stay, respectively. The models we investigate include variations on logistic regression (with PCA preprocessing and regularization), random forests, support vector classifiers, and a handful of deep learning models, including a multi-input model that takes as input both the numeric data consisting of hourly observations as well as free-form text data from chart notes. In each case, we tune hyperparameters through cross-validation and report the results on a test datasets.

The MIMIC III database consists of deidentified health-related data associated to over 40,000 patients who stayed in critical care units of the Beth Israel Deaconess Medical Center between 2001 and 2012. The data was accessed through Physionet, and though publicly available, requires credentialing before access. As such, we do not display the data itself in this document, only the results of our models and the code used to run them, which may be replicated by anyone who has the data. For the first step of processing the data, we use the MIMIC-Extract open-source pipeline <cite data-cite="11046887/I5YVQAME"></cite>, which handles unit conversion, removal of outliers, and accounts for duplication in the original dataset. The remaining processing needed for our models is carried out by the [processing.py](processing.py) script.

We begin by summarizing the structure of our data and the metrics by which we will evaluate the performance of our models. We then present a summary and discussion of our results. The code for running our models and tuning hyperparameters is presented in the final sections.

# The MIMIC III dataset

## Structure of the data

Our starting point is the output of the MIMIC-Extract pipeline, which handles unit conversion, removal of outliers, and accounts for duplication in the original MIMIC III dataset. For each patient's ICU stay, the output of the MIMIC-Extract pipeline contains mean hourly measurements of 104 features, along with categorical variables encoding age, ethnicity, and gender. For many patients and hours, these features are missing, and we begin by further processing the MIMIC-Extract data by applying the following steps (carried out in the file [processing.py](processing.py)).

- We impute the missing measurements following the "simple imputation" scheme applied in <cite data-cite="11046887/J3W4HI5G"></cite> and also used in <cite data-cite="11046887/I5YVQAME"></cite>, whose code we adapt for this part. For each feature, we first forward-fill missing measurements, then for the remaining missing measurements (those with no prior measurements) we replace them with the individual mean (or, in the case of no measurements of that feature, we fill it by the population mean).

- We encode the categorical variables--age (divided into 10 buckets by quantile), ethnicity, and gender--by dummy variables.

- We append to each patient an additional string variable consisting of the unstructured chart notes for that patient over the relevant time window. Unlikely the hourly observation data, we do not keep track of the time series nature of the chart note data at present.

- We construct three datasets, one for each window size of 12, 24, and 48 hours. We include a 6 hour buffer so that patients included in the 12, 24, and 48 hour datasets were in the ICU for at least 18, 30, and 54 hours, respectively.

- We split the data into a training set and test set (80% and 20%, respectively) with stratification to ensure that the mortality rate in the training and test sets is similar. We split the data in such a way that the 12, 24, and 48 hour training and test sets are nested, so that, for instance, a patient in the 24 hour training set appears in the 12 hour training set and not the 12 hour test set.

For now, we focus only on the hourly observations and categorical features, leaving the unstructured chart note data for our neural network models in the second half of this document.

## Reading and processing the data

In [19]:
import os
from IPython.display import display
import sys
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dill
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import metrics, svm
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV, PredefinedSplit, cross_validate, train_test_split, StratifiedKFold
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve, roc_auc_score, f1_score, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix, average_precision_score
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.cross_decomposition import PLSRegression
from joblib import parallel_backend
from IPython.display import display
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasClassifier
import keras
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, GRU, Bidirectional, Conv1D,MaxPooling1D, BatchNormalization, Flatten,Embedding
from keras.layers.experimental.preprocessing import Normalization
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras import Input

We read in the train and test data corresponding to the three window sizes.

In [18]:
os.chdir("/Volumes/Seagate/physionet")

X_train_12_total=pd.read_csv("X_train_12.csv", index_col=[0, 1])
y_train_12=pd.read_csv("y_train_12.csv", index_col=[0,1])['mort_icu'].values
X_train_24_total=pd.read_csv("X_train_24.csv", index_col=[0, 1])
y_train_24=pd.read_csv("y_train_24.csv", index_col=[0,1])['mort_icu'].values
X_train_48_total=pd.read_csv("X_train_48.csv", index_col=[0, 1])
y_train_48=pd.read_csv("y_train_48.csv", index_col=[0,1])['mort_icu'].values

X_test_12_total=pd.read_csv("X_test_12.csv", index_col=[0, 1])
y_test_12=pd.read_csv("y_test_12.csv", index_col=[0,1])['mort_icu'].values
X_test_24_total=pd.read_csv("X_test_24.csv", index_col=[0, 1])
y_test_24=pd.read_csv("y_test_24.csv", index_col=[0,1])['mort_icu'].values
X_test_48_total=pd.read_csv("X_test_48.csv", index_col=[0, 1])
y_test_48=pd.read_csv("y_test_48.csv", index_col=[0,1])['mort_icu'].values

X_train_12=X_train_12_total.drop("TEXT", axis=1)
X_train_24=X_train_24_total.drop("TEXT", axis=1)
X_train_48=X_train_48_total.drop("TEXT", axis=1)

X_test_12=X_test_12_total.drop("TEXT", axis=1)
X_test_24=X_test_24_total.drop("TEXT", axis=1)
X_test_48=X_test_48_total.drop("TEXT", axis=1)

X_train_text_12=X_train_12_total[["TEXT"]]
X_train_text_24=X_train_24_total[["TEXT"]]
X_train_text_48=X_train_48_total[["TEXT"]]

X_test_text_12=X_test_12_total[["TEXT"]]
X_test_text_24=X_test_24_total[["TEXT"]]
X_test_text_48=X_test_48_total[["TEXT"]]

trainXs=[X_train_12, X_train_24, X_train_48]
testXs=[X_test_12, X_test_24, X_test_48]
Xs=trainXs.append(testXs)
trainYs=[y_train_12, y_train_24, y_train_48]
testYs=[y_test_12, y_test_24, y_test_48]
Ys=trainYs.append(testYs)

#dill.dump_session("/Volumes/Seagate/physionet/6_21_loaded")

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/Seagate/physionet'

In [17]:
#dill.load_session("/Volumes/Seagate/physionet/6_21_loaded")
cols_all=[X_train_12.columns, X_train_24.columns, X_train_48.columns]
ethnicity=[cols[cols.str.contains("ethnicity_")].values for cols in cols_all]
age=[cols[cols.str.contains("age_")].values for cols in cols_all]
gender=[cols[cols.str.contains("gender_")].values for cols in cols_all]
num_features=[cols[cols.str.contains("_mean_")].values for cols in cols_all]
cat_features=[np.concatenate((age[i], gender[i], ethnicity[i])) for i in [0,1,2]]
dtypes_cat = [{k:"category" for k in cat_features[i]} for i in [0, 1,2]]
dtypes_num=[{k:"float64" for k in num_features[i]} for i in [0, 1,2]]
dtypes=[dict(dtypes_cat[i], **dtypes_num[i]) for i in [0, 1,2]]

for i in [0, 1, 2]:
    trainXs[i]=trainXs[i].astype(dtypes[i])
    testXs[i]=testXs[i].astype(dtypes[i])

NameError: name 'X_train_12' is not defined

We separate the unstructured chart note data from the rest of the data. We will come back to the chart data when we apply our word embedding models.

In [3]:
X_train_text_12=X_train_text_12[["TEXT"]].astype(str)
X_train_text_24=X_train_text_24[["TEXT"]].astype(str)
X_train_text_48=X_train_text_48[["TEXT"]].astype(str)
X_test_text_12=X_test_text_12[["TEXT"]].astype(str)
X_test_text_24=X_test_text_24[["TEXT"]].astype(str)
X_test_text_48=X_test_text_48[["TEXT"]].astype(str)

text_train_pairs=[(X_train_text_12, y_train_12), (X_train_text_24, y_train_24), (X_train_text_48, y_train_48)]
text_test_pairs=[(X_test_text_12, y_test_12), (X_test_text_24, y_test_24), (X_test_text_48, y_test_48)]

For each window size, we also construct two versions of the data: one with all features, and one with only numerical features (excluding the dummy variables representing age, ethnicity and gender).

In [4]:
## Numerical features only
X_train_12a=trainXs[0][num_features[0]].copy()
X_train_24a=trainXs[1][num_features[1]].copy()
X_train_48a=trainXs[2][num_features[2]].copy()

X_test_12a=testXs[0][num_features[0]].copy()
X_test_24a=testXs[1][num_features[1]].copy()
X_test_48a=testXs[2][num_features[2]].copy()

train_pairs_a=[(X_train_12a, y_train_12), (X_train_24a, y_train_24), (X_train_48a, y_train_48)]
train_pairs=[(X_train_12, y_train_12), (X_train_24, y_train_24), (X_train_48, y_train_48)]

test_pairs_a=[(X_test_12a, y_test_12), (X_test_24a, y_test_24), (X_test_48a, y_test_48)]
test_pairs=[(X_test_12, y_test_12), (X_test_24, y_test_24), (X_test_48, y_test_48)]


train_pairs_total=np.concatenate((train_pairs_a, train_pairs))
test_pairs_total=np.concatenate((test_pairs_a, test_pairs))
train_pairs_12=[(X_train_12a, y_train_12), (X_train_12, y_train_12)]
train_pairs_24=[(X_train_24a, y_train_24), (X_train_24, y_train_24)]
train_pairs_48=[(X_train_48a, y_train_48), (X_train_48, y_train_48)]

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


We tune hyperparameters on our training data and only use the test data for the final step of model evaluation. We also note that in splitting the train and test data in the [procesing.py](processing.py) script, we make a point to use a stratified splitting, which ensures the same percentage of positives in the train set as in the test set. Because the two classes in the data are so imbalanced (see below), this turns out to make a big difference, and early attempts to evaluate models that did not use stratification in splitting the data ran into stability issues.

# Discussion of metrics

Our goal in this document is mortality prediction, and we begin by noting that our binary classification problem is imbalanced. Indeed, as a simple baseline for our models, we calculate the percent of patients who did not die during their ICU stay.

In [128]:
print(1-y_train_12.sum()/y_train_12.shape[0])
print(1-y_train_24.sum()/y_train_24.shape[0])
print(1-y_train_48.sum()/y_train_48.shape[0])

0.9368238531418448
0.928935043779271
0.9089649198787354


A trivial model predicting survival for each patient would have around 93.7% accuracy on the 12 hour dataset, 92.9% accuracy for the 24 hour dataset, and 90.9% accuracy on the 48 hour dataset. This is first baseline against which we compare our subsequent models.

To get a more sophisticated view of the predictive power of our model, we keep track of the following metrics.

- **Accuracy**: percent of ICU stays in which we predicted mortality correctly.

- **AUROC**: area under the ROC curve, which plots false positive rate against true positive rate.

- **AUPR**: area under the precision recall curve, which plots recall (percent of true positives which are predicted positives) against precision (percent of predicted positives which are true positives).

- **F1 score**: the harmonic mean of Precision and Recall.

Because of the low mortality rate in our data, our classes are quite imabalanced. Furthermore, we are in a setting where it is very likely that a false negative (failing to predict mortality for a patient who will die) carries a higher risk than a false positive (predicting mortality for a patient who will not die). As such, evaluating our models based on the AUROC score alone is somewhat misleading as the AUROC score may be overly optimistic in this setting (see <cite data-cite="11046887/C5V6RWQ5"></cite> for an example). We attach more significance to the area under the Precision-Recall curve, AUPR, which frames our problem in terms of the tradeoff between precision and recall: the challenge is to produce sufficiently high recall (i.e. to correctly detect patients who will die) while maintaining sufficiently high precision (identifying patients who will die correctly).

We define our metrics and write two auxilliary functions, one for evaluating hyperparameters by cross-validation, and the other for final evaluation of our models on the test data.

In [5]:
scorers={'accuracy': make_scorer(accuracy_score), 'precision': make_scorer(precision_score),
         'recall': make_scorer(recall_score),'roc_auc': make_scorer(roc_auc_score, needs_proba=True),
         'average_precision': make_scorer(average_precision_score), 'f1': make_scorer(f1_score)}

In [12]:
def model_eval(model, X_train, y_train, param_grid={}, scoring=scorers, refit="roc_auc"):
    if param_grid=={}:
        return pd.DataFrame(cross_validate(model, X_train, y_train, scoring=scoring)).mean()
    else:
        fit_grid=GridSearchCV(model, param_grid, scoring=scoring, refit=refit, verbose=4).fit(X_train, y_train)
        return pd.DataFrame(fit_grid.cv_results_)

In [92]:
def model_score(model, X_train, y_train, X_test, y_test):
    fit=model.fit(X_train, y_train)
    y_pred=model.predict(X_test)
    y_prob=model.predict_proba(X_test)[:,1]
    acc=accuracy_score(y_test, y_pred)
    rec=recall_score(y_test, y_pred)
    prec=precision_score(y_test, y_pred)
    auroc=roc_auc_score(y_test, y_prob)
    ap=average_precision_score(y_test, y_prob)
    f1=f1_score(y_test, y_pred)
    score={'accuracy':acc, 'recall':rec, 'precision':prec, 'auroc':auroc, 'average precision':ap, 'f1':f1}
    return score
    

# Summary of results and discussion

## Summary of models evaluated

In this project, we evaluate the following models on each of our 12, 24, and 48 hour datasets.

- **Logistic regression** (with and without categorical features)  


- **Logistic regression with Principal Component Analysis (PCA) preprocessing**. We use cross-validation to select the optimal number of principal components.


- **Logistic regression with regularization**. We evaluate both L1 (lasso) and L2 (ridge) regularization at several values of the regularization parameter to select optimal regularization.  


- **Random forests**. We cross-validate to optimize the number of estimators.  


- **Support Vector Classifiers**. We cross-validate to select the optimal choice of kernel (choosing between linear, polynomial, radial basis functions, and sigmoid), then for the optimal kernel, we cross-validate to select the optimal values of C and gamma.  


- **Multilayer perceptron on structured ICU stay data**. We tested a variety of architectures to settle on one with 3 dense hidden layers (with regularization and dropout). We also experimented with LSTM and GRU layers to account for the time-series nature of our observations, but we exclude these from our analysis since we were unable to achieve optimal results with them (see the Future Work section for more on this).  


- **Neural network on chart note data** To process the input data, we experimented with both word emebeddings (with GRU and 1D Convolutional layer built on top) as well as a simple TF-IDF encoding of the data. We achieved the strongest performance with the TF-IDF encoding of the chart note data followed by a multilayer perceptron with a single dense hidden layer.


- **Multi-input neural network** We combined both of the above neural nets to construct one with two inputs, each fed through dense layers before being concatenated and fed through one more dense hidden layer.

## Results

The table below summarizes the results for each of our models with the optimal choice of hyperparameters (selected through cross-validation), evaluated on our test dataset.

| Model  | Window | Hyperparameters  | Accuracy  | AUROC | AUPR | F1 |
|---|---|---|---|---|---|---|
|Logistic regression (LR)| 12 hr |-|0.942|0.879|0.439|0.384|
|| 24 hr |- |0.930|0.846|0.408|0.397|
|| 48 hr |-|0.890|0.794|0.375|0.347|
|LR w/ PCA| 12 hr |n_components=300|0.945|0.893|0.477|0.378|
|| 24 hr | n_components=300|0.939|0.879|0.480|0.405|
|| 48 hr |n_components=300|0.921|0.873|0.513|0.407|
|LR regression w/ Regularization | 12 hr | L1, C=1.0 |0.944|0.893|0.470|0.355|
|| 24 hr |L2, C=1.0  |0.939|0.879|0.473|0.368|
|| 48 hr | L2, C=0.1 |0.923|0.870|0.510|0.412|
|Random Forests | 12 hr | n_estimators=1800 |0.943|0.892|0.493|0.454|
|| 24 hr | n_estimators=1400 |0.939|0.884|0.501|0.253|
|| 48 hr |n_estimators=400 |0.929|0.876|0.505|0.298|
|Support Vector Classifier | 12 hr | kernel=rbf, gamma=0.000, C=0.056|0.193|0.882|0.388|0.314|
|| 24 hr | kernel=rbf, gamma=0.000, C=0.056 |0.742|0.868|0.364|0.312|
|| 48 hr |kernel=rbf, gamma=0.000, C=0.056 |0.689|0.828|0.336|0.308
|NN w/ numerical input | 12 hr |(see below)|0.946|0.913|0.507|0.422|
|| 24 hr |- |0.948|0.902|0.528|0.456|
|| 48 hr |-|0.948|0.864|0.528|0.469|
|NN w/ chart note data input| 12 hr |- |0.945|0.883|0.452|0.319|
|| 24 hr |-  |0.933|0.858|0.377|0.151|
|| 48 hr |- |0.915|0.811|0.302|0.047|
|Multi-input NN| 12 hr | |0.948|**0.927**|**0.576**|0.378|
|| 24 hr |-  |0.948|**0.917**|**0.555**|0.416|
|| 48 hr |- |0.948|**0.889**|**0.580**|0.485|

## Discussion

For each window size, we find the strongest performance (in terms of both AUROC as well as AUPR) is achieved by the multi-input neural network model. This is the model which takes advantage of all of our data (the hourly observations, the categorical variables, and the chart notes). Compared to the neural net model which uses only the hourly observations, we see that incorporating the unstructured chart notes (via TF-IDF encoding) results in a modest increase in both AUROC and AUPR.

We also note that due to the class imbalance in our problem, the AUROC score is consistently and substantially larger than the AUPR score. Indeed, achieving high recall at a high precision is quite challenging. For instance, the multi-input neural network model on the 12 hour data with the highest AUROC score (0.927) has an AUPR score of 0.576. At a threshhold of 0.5, this corresponds to a recall of only 0.251 with a precision of 0.766. The model on the 48 hour data which achieves the highest AUPR score overall of 0.580 has, at a 0.5 threshhold, a recall of 0.375 with a precision of 0.687. We see that AUPR gives a more accurate assesment of our models as AUROC is somewhat overoptimistic.

It is also interesting to note that on our best model as well as several others, we find that the highest AUROC score is achieved with a 12 hour window size, while the highest AUPR score is achieved with a 48 hour window size. We expect that there are two competing factors that come with increasing the window size: on the one hand, mortality prediction may be more difficult for patients who have been in the ICU longer, and on the other hand, using a larger window size provides us with more data to make predictions.

Besides for the deep learning models, the following three models all achieved comparable performance: random forests, logistic regression with PCA preprocessing, and logistic regression with regularization. If interpretability is of importance to our model, the logistic regression models provide this with only a modest (10-20%, depending on window size) drop in AUPR. 

Our multi-input neural network model trained on both hourly observation, categorical, and chart note data performed favorably compared to the strongest model benchmarked in <cite data-cite="11046887/I5YVQAME"></cite> in the in-ICU mortality prediction task at a window size of 24 hours, which had an AUROC and AUPR scores of 89.1 and 50.9, respectively (compared to 91.7 and 55.5 in our case). We note that our best model had the advantage of being trained on the chart note data in addition to the hourly observations and categorical variables. This adds further evidence to the observation based on our models that incorporating unstructured chart note data from electronic health records improves predictive power. However, an important caviat is that we used stratified splitting in constructing our training and test sets to ensure that the mortality rates were similar in the two, whereas <cite data-cite="11046887/I5YVQAME"></cite> did not. We observed that the splitting into train and test sets without stratification resulted in somewhat unstable mortality rates between the two, which in turn effected the scoring of our models. It is thus likely that the comparison between our results and those benchmarked in <cite data-cite="11046887/I5YVQAME"></cite> is not entirely valid.

Finally, we note that in processing the unstructured chart note data, we took the simplest approach in concatenated all chart notes recorded during the window of interest and encoding these using TF-IDF (Text Frequency-Inverse Document Frequency). This approach only keeps track of which words appear in which chart notes and with what frequency and does not depend on the syntactic structure of the chart notes or on what words appear near other words and in what order. We did initially experiment with embedding layers in our chart note models, which are potentially more sensitive to this kind of structure; however, we found that they did not yield any visible improvement compared to the simple TF-IDF models, suggesting that, at first approximation, perhaps the TF-IDF encoding captures the most important features of the chart notes and that the other structural properties of the chart notes are less important. Whether one could use more sophisticated models based on word embeddings to achieve better results is an interesting question worth further investigation.

# Future work

Since deep learning models achieved the best results on our data, it is natural to expect that more sophisticated neural network architectures might achieve better results. Likely the biggest limitation of our neural network models is that they don't take full advantage of the time series nature of the data, or of the fact that many of observations were missing and had to be imputed. In experimenting with various models, we did do some preliminary evaluation of models with LSTM and GRU layers that could better exploit the time series nature of our hourly observations. However, we omit these from our analysis from the time being, as we have been unable to achieve better results than the models discussed above. However, we do note that <cite data-cite="11046887/J3W4HI5G"></cite> found that such models do lead to improvement in model performance.

Thus, a more optimal approach would take into account the fact that the original dataset from MIMIC-Extract has a substantial amount of missing data, that the missingness of data is in fact biased, and as such, taking into account which observations are missing and the time since the last observation could achieve stronger results than the imputation we used above. Indeed, this is the idea behind the GRU-D (GRU with Decay) layer developed and implemented for the MIMIC data in <cite data-cite="11046887/J3W4HI5G"></cite>. The GRU-D layer functions similarly to a standard GRU layer but includes decay terms related to the time since last observation and a variable indicating whether the observation was missing and imputed. 

A particularly promising area for future work is combining our hybrid structured-unstructured input model with one with GRU-D type layers. In the case of textual chart data, it makes sense to record the time since the last observation and to incorporate a decay term, just as the GRU-D layer do with the numerical hourly observations. Such a model might require development of a 2-dimensional GRU-D layer (one direction for the text and another for time at which notes were recorded).

# Model specification, tuning, and evaluation

The remainder of the document contains our code for specifying the models described above, tuning hyperparameters, and evaluating the models on the test set.

## Linear models

### Logistic regression

As a standard, well-developed baseline model, we begin by running a simple logistic regression for each of our models. We would like to explore whether our models do better with input just the numerical hourly observations or if the dummy variables tracking the static features (age, ethnicity, and gender) add predictive value. As such, for each of the three window sizes, we run our model on both sets of input data.

In [87]:
with parallel_backend('threading', n_jobs=4):
    lm=LogisticRegression(max_iter=500, verbose=1, n_jobs=-1)
    ct_scale=[ColumnTransformer([('scale', StandardScaler(), num_features[i])], remainder="passthrough") for i in [0,1, 2]]
    lm_pipe=[Pipeline(steps=[('scale', ct_scale[i]), ('logistic', lm)]) for i in [0, 1, 2]]
    a=time.perf_counter()
    lm_results_12=[model_eval(lm_pipe[0], X, y) for X, y in train_pairs_12]
    lm_results_24=[model_eval(lm_pipe[1], X, y) for X, y in train_pairs_24]
    lm_results_48=[model_eval(lm_pipe[1], X, y) for X, y in train_pairs_48]
    print(time.perf_counter()-a)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.5min finished
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.5min finished
/opt/anaconda3/lib/p

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.8min finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please als

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.2min finished
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-l

967.1746828809992


In [31]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
display(lm_results_12)
display(lm_results_24)
display(lm_results_48)

[fit_time                  71.706088
 score_time                 0.984203
 test_accuracy              0.936975
 test_precision             0.504102
 test_recall                0.247619
 test_roc_auc               0.843349
 test_average_precision     0.172127
 test_f1                    0.329363
 dtype: float64,
 fit_time                  71.656853
 score_time                 0.981865
 test_accuracy              0.938369
 test_precision             0.528254
 test_recall                0.268500
 test_roc_auc               0.853034
 test_average_precision     0.187233
 test_f1                    0.353096
 dtype: float64]

[fit_time                  103.156462
 score_time                  1.763528
 test_accuracy               0.922470
 test_precision              0.432205
 test_recall                 0.290115
 test_roc_auc                0.809527
 test_average_precision      0.175958
 test_f1                     0.344356
 dtype: float64,
 fit_time                  105.652200
 score_time                  1.512698
 test_accuracy               0.923539
 test_precision              0.446935
 test_recall                 0.307307
 test_roc_auc                0.819065
 test_average_precision      0.186231
 test_f1                     0.361558
 dtype: float64]

[fit_time                  606.249958
 score_time                  2.567601
 test_accuracy               0.913729
 test_precision              0.588415
 test_recall                 0.160880
 test_roc_auc                0.780618
 test_average_precision      0.174458
 test_f1                     0.250443
 dtype: float64,
 fit_time                  156.201641
 score_time                  3.690031
 test_accuracy               0.913729
 test_precision              0.598771
 test_recall                 0.149461
 test_roc_auc                0.779856
 test_average_precision      0.170169
 test_f1                     0.236231
 dtype: float64]

We see that for each window, our model performs slightly better when trained on both numerical and categorical features. As such, we use both sets of features going forward. We now evaluate our linear model on the test set.

In [ ]:
lm_pipe=[Pipeline(steps=[('scale', ct_scale[i]), ('logistic', lm)]) for i in [0, 1, 2]]
lm_scores=[model_score(lm_pipe[i], train_pairs[i][0], train_pairs[i][1], test_pairs[i][0], test_pairs[i][1]) for i in [0, 1, 2]]

In [91]:
lm_scores

[{'accuracy': 0.9428614503241369,
  'recall': 0.28162291169451076,
  'precision': 0.6020408163265306,
  'auroc': 0.8788608062631688,
  'average precision': 0.43947197588479703,
  'f1': 0.38373983739837403},
 {'accuracy': 0.9295172132817274,
  'recall': 0.3333333333333333,
  'precision': 0.49137931034482757,
  'auroc': 0.8461259710841953,
  'average precision': 0.40817232280722743,
  'f1': 0.397212543554007},
 {'accuracy': 0.8895104895104895,
  'recall': 0.35,
  'precision': 0.3442622950819672,
  'auroc': 0.7938676844783714,
  'average precision': 0.3753676491740862,
  'f1': 0.3471074380165289}]

### Logistic regression with PCA

We expect that some of our covariates are highly correlated given that our imputation method results in observations at missing times being filled in by the observations from the last time they were recorded. Thus, a model that takes this into account would do better, and we investigate two approaches in this direction. The first method, Principal Component Analysis (PCA) focuses only on the input space and projects the data on the k-dimensional subspace which best captures the variance in the input space. We use cross-validation to select the optimal value of k (the number of principal components), selecting from multiples of 100. We find in each case that 300 principal components achieves the optimal average precision (and AUROC), far fewer than the total number of features in the data.

In [46]:
a=time.perf_counter()
def PCA_params(b):
    return {'pca__n_components':list(range(100,b,100))}

ct_scale=[ColumnTransformer([('scale', StandardScaler(), num_features[i])], remainder="passthrough") for i in [0,1, 2]]


logpca_pipe=[Pipeline(steps=[('scale', ct_scale[i]),
                             ('pca', PCA()),
                            ('logistic', LogisticRegression(max_iter=500, verbose=1, n_jobs=-1))]) for i in [0, 1, 2]]
                             
a=time.perf_counter()
logpca_results=[model_eval(logpca_pipe[i], train_pairs[i][0], train_pairs[i][1],
                        param_grid=PCA_params(train_pairs[i][0].shape[1])) for i in [0, 1, 2]]
print(time.perf_counter()-a)

Fitting 5 folds for each of 14 candidates, totalling 70 fits
[CV] pca__n_components=100 ...........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    8.8s finished


[CV]  pca__n_components=100, accuracy=0.940, average_precision=0.181, f1=0.333, precision=0.564, recall=0.236, roc_auc=0.887, total=  14.9s
[CV] pca__n_components=100 ...........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   14.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    7.3s finished


[CV]  pca__n_components=100, accuracy=0.943, average_precision=0.198, f1=0.346, precision=0.630, recall=0.239, roc_auc=0.857, total=  12.5s
[CV] pca__n_components=100 ...........................................


...

[CV]  pca__n_components=5100, accuracy=0.882, average_precision=0.193, f1=0.370, precision=0.359, recall=0.381, roc_auc=0.761, total= 2.0min
[CV] pca__n_components=5100 ..........................................


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   30.4s finished


[CV]  pca__n_components=5100, accuracy=0.881, average_precision=0.168, f1=0.324, precision=0.337, recall=0.313, roc_auc=0.735, total= 2.0min


[Parallel(n_jobs=1)]: Done 255 out of 255 | elapsed: 425.9min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    6.7s finished


33126.441272868


In [84]:
logpca_results[0]

,Unnamed: 0,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_pca__n_components,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall,split0_test_roc_auc,split1_test_roc_auc,split2_test_roc_auc,split3_test_roc_auc,split4_test_roc_auc,mean_test_roc_auc,std_test_roc_auc,rank_test_roc_auc,split0_test_average_precision,split1_test_average_precision,split2_test_average_precision,split3_test_average_precision,split4_test_average_precision,mean_test_average_precision,std_test_average_precision,rank_test_average_precision,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1
0,0,13.377839,2.702216,0.365000,0.044609,100,{'pca__n_components': 100},0.940256,0.943083,0.943837,0.939879,0.939303,0.941272,0.001828,3,0.564286,0.629921,0.707865,0.589474,0.594203,0.617150,0.049959,1,0.235821,0.238806,0.188060,0.166667,0.122388,0.190348,0.043819,14,0.886974,0.857137,0.854827,0.852360,0.843760,0.859011,0.014695,5,0.181318,0.198488,0.184384,0.151016,0.128143,0.168670,0.025495,14,0.332632,0.346320,0.297170,0.259861,0.202970,0.287791,0.051960,14
1,1,22.541213,4.981858,0.378760,0.045489,200,{'pca__n_components': 200},0.942141,0.943460,0.943272,0.940256,0.939114,0.941649,0.001704,1,0.586420,0.625899,0.670000,0.582609,0.575000,0.607986,0.035676,2,0.283582,0.259701,0.200000,0.199405,0.137313,0.216000,0.051381,13,0.891545,0.865947,0.863489,0.853318,0.846573,0.864174,0.015365,3,0.211530,0.209287,0.184509,0.166872,0.133432,0.181126,0.029006,13,0.382294,0.367089,0.308046,0.297118,0.221687,0.315247,0.057126,13
2,2,17.102243,4.763905,0.521843,0.094651,300,{'pca__n_components': 300},0.940068,0.943649,0.944591,0.939879,0.939868,0.941611,0.002071,2,0.546448,0.609756,0.684685,0.560284,0.586957,0.597626,0.048690,3,0.298507,0.298507,0.226866,0.235119,0.161194,0.244039,0.051325,12,0.899435,0.874975,0.868564,0.853415,0.855828,0.870444,0.016542,1,0.207408,0.226306,0.204144,0.180169,0.147583,0.193122,0.027086,1,0.386100,0.400802,0.340807,0.331237,0.252927,0.342375,0.051871,12
3,3,14.611945,0.878609,0.402928,0.006906,400,{'pca__n_components': 400},0.937995,0.942329,0.942141,0.941387,0.939680,0.940706,0.001647,4,0.514151,0.585799,0.607692,0.588652,0.572816,0.573822,0.031850,4,0.325373,0.295522,0.235821,0.247024,0.176119,0.255972,0.051465,11,0.899272,0.868498,0.872309,0.844821,0.852778,0.867536,0.018786,2,0.209884,0.217595,0.191554,0.193093,0.152910,0.193007,0.022357,2,0.398537,0.392857,0.339785,0.348008,0.269406,0.349719,0.046467,10
4,4,18.254330,0.742101,0.492885,0.051012,500,{'pca__n_components': 500},0.938183,0.938372,0.941576,0.942518,0.938360,0.939802,0.001858,5,0.515837,0.521978,0.595420,0.604027,0.537736,0.555000,0.037309,5,0.340299,0.283582,0.232836,0.267857,0.170149,0.258945,0.056340,10,0.891040,0.857872,0.863992,0.843256,0.849610,0.861154,0.016526,4,0.217190,0.193255,0.187071,0.208156,0.143899,0.189914,0.025351,4,0.410072,0.367505,0.334764,0.371134,0.258503,0.348396,0.050897,11
5,5,22.752256,3.287990,0.541512,0.089785,600,{'pca__n_components': 600},0.935922,0.938372,0.941764,0.939502,0.939114,0.938935,0.001884,7,0.489177,0.521505,0.595588,0.549669,0.549180,0.541024,0.035178,6,0.337313,0.289552,0.241791,0.247024,0.200000,0.263136,0.046694,9,0.877603,0.852873,0.860714,0.841970,0.847912,0.856214,0.012335,6,0.206846,0.195858,0.191878,0.183463,0.160354,0.187680,0.015597,6,0.399293,0.372361,0.343949,0.340862,0.293217,0.349936,0.035424,9
6,6,23.305138,0.506238,0.577206,0.044623,700,{'pca__n_components': 700},0.937618,0.936487,0.942329,0.940445,0.938549,0.939086,0.002077,6,0.50

In [86]:
logpca_results[1]

,Unnamed: 0,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_pca__n_components,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall,split0_test_roc_auc,split1_test_roc_auc,split2_test_roc_auc,split3_test_roc_auc,split4_test_roc_auc,mean_test_roc_auc,std_test_roc_auc,rank_test_roc_auc,split0_test_average_precision,split1_test_average_precision,split2_test_average_precision,split3_test_average_precision,split4_test_average_precision,mean_test_average_precision,std_test_average_precision,rank_test_average_precision,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1
0,0,13.402088,3.947793,0.540392,0.142627,100,{'pca__n_components': 100},0.936371,0.932807,0.937134,0.933062,0.931772,0.934229,0.002119,2,0.609023,0.567568,0.677778,0.610390,0.603774,0.613706,0.035681,1,0.290323,0.225806,0.218638,0.167857,0.114695,0.203464,0.059028,26,0.871954,0.858386,0.854258,0.830539,0.839161,0.850859,0.014581,5,0.227208,0.183136,0.203673,0.161761,0.132132,0.181582,0.032895,26,0.393204,0.323077,0.330623,0.263305,0.192771,0.300596,0.067823,26
1,1,21.527019,2.490811,0.685529,0.163570,200,{'pca__n_components': 200},0.935098,0.936880,0.934843,0.935607,0.930754,0.934636,0.002064,1,0.582192,0.616541,0.607477,0.658824,0.552239,0.603454,0.035576,2,0.304659,0.293907,0.232975,0.200000,0.132616,0.232832,0.063262,25,0.884973,0.868057,0.873775,0.842215,0.822170,0.858238,0.022848,2,0.226747,0.231346,0.195994,0.188777,0.134845,0.195542,0.034600,8,0.400000,0.398058,0.336788,0.306849,0.213873,0.331114,0.068692,25
2,2,14.621874,2.452525,0.660526,0.127907,300,{'pca__n_components': 300},0.936625,0.936116,0.932807,0.932044,0.931008,0.933720,0.002244,3,0.591463,0.595890,0.555556,0.565657,0.555556,0.572824,0.017477,3,0.347670,0.311828,0.268817,0.200000,0.143369,0.254337,0.074152,24,0.886923,0.873047,0.881071,0.833598,0.819908,0.858910,0.026972,1,0.251956,0.234683,0.201264,0.170143,0.140495,0.199708,0.040831,6,0.437923,0.409412,0.362319,0.295515,0.227920,0.346618,0.076455,23
3,3,16.563285,2.066983,0.676404,0.102147,400,{'pca__n_components': 400},0.936116,0.933316,0.932044,0.929753,0.929481,0.932142,0.002448,5,0.582353,0.553459,0.541096,0.518868,0.512195,0.541594,0.025231,5,0.354839,0.315412,0.283154,0.196429,0.150538,0.260074,0.075663,23,0.884354,0.868820,0.878149,0.827666,0.815236,0.854845,0.027988,3,0.252455,0.223181,0.204117,0.159187,0.137441,0.195276,0.041889,9,0.440980,0.401826,0.371765,0.284974,0.232687,0.346446,0.076635,24
4,4,18.619445,1.572313,0.684735,0.078352,500,{'pca__n_components': 500},0.932553,0.937643,0.932553,0.930262,0.929990,0.932600,0.002747,4,0.539773,0.604938,0.546667,0.525424,0.523256,0.548011,0.029777,4,0.340502,0.351254,0.293907,0.221429,0.161290,0.273676,0.072462,22,0.868698,0.872923,0.878811,0.823595,0.820179,0.852841,0.025500,4,0.230625,0.258555,0.210809,0.171829,0.143968,0.203157,0.040907,1,0.417582,0.444444,0.382284,0.311558,0.246575,0.360489,0.072324,21
5,5,21.667192,1.725889,0.707858,0.054520,600,{'pca__n_components': 600},0.929753,0.933825,0.931280,0.928735,0.929990,0.930717,0.001753,6,0.507692,0.554913,0.529412,0.500000,0.518868,0.522177,0.019166,6,0.354839,0.344086,0.290323,0.235714,0.197133,0.284419,0.060902,21,0.873753,0.866391,0.874066,0.812047,0.812817,0.847815,0.029021,6,0.225962,0.237515,0.204095,0.172324,0.159312,0.199842,0.030065,5,0.417722,0.424779,0.375000,0.320388,0.285714,0.364721,0.054275,7
6,6,25.472933,1.068412,0.846003,0.110857,700,{'pca__n_components': 700},0.928990,0.933316,0.931026,0.929499,0.930244,0.930615,0.001516,7,0.50

In [85]:
logpca_results[2]

,Unnamed: 0,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_pca__n_components,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall,split0_test_roc_auc,split1_test_roc_auc,split2_test_roc_auc,split3_test_roc_auc,split4_test_roc_auc,mean_test_roc_auc,std_test_roc_auc,rank_test_roc_auc,split0_test_average_precision,split1_test_average_precision,split2_test_average_precision,split3_test_average_precision,split4_test_average_precision,mean_test_average_precision,std_test_average_precision,rank_test_average_precision,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1
0,0,9.535711,0.681575,0.469729,0.078217,100,{'pca__n_components': 100},0.920312,0.915115,0.918579,0.918146,0.909052,0.916241,0.003966,3,0.630000,0.581395,0.644737,0.723404,0.508197,0.617547,0.071224,1,0.300000,0.238095,0.233333,0.161905,0.146919,0.216051,0.055755,51,0.861542,0.840695,0.837936,0.808210,0.819011,0.833479,0.018479,5,0.252664,0.207722,0.220166,0.193346,0.152620,0.205303,0.032828,10,0.406452,0.337838,0.342657,0.264591,0.227941,0.315896,0.062875,51
1,1,15.867137,2.587056,0.520278,0.032120,200,{'pca__n_components': 200},0.923343,0.922477,0.915981,0.918579,0.911217,0.918320,0.004439,2,0.625954,0.642202,0.575472,0.661765,0.541667,0.609412,0.044342,2,0.390476,0.333333,0.290476,0.214286,0.184834,0.282681,0.075496,50,0.879135,0.865555,0.863048,0.837657,0.822067,0.853493,0.020653,1,0.299855,0.274700,0.231691,0.213266,0.174610,0.238824,0.044361,3,0.480938,0.438871,0.386076,0.323741,0.275618,0.381049,0.074494,10
2,2,13.077137,0.442092,0.726832,0.010441,300,{'pca__n_components': 300},0.924210,0.922910,0.919879,0.914682,0.911217,0.918579,0.004930,1,0.624113,0.633333,0.610619,0.573034,0.534091,0.595038,0.036752,3,0.419048,0.361905,0.328571,0.242857,0.222749,0.315026,0.073383,49,0.876710,0.872089,0.863965,0.837959,0.812606,0.852666,0.024100,2,0.314370,0.287240,0.261698,0.208026,0.189995,0.252266,0.046913,1,0.501425,0.460606,0.427245,0.341137,0.314381,0.408959,0.070847,1
3,3,16.226363,0.354436,0.818840,0.090619,400,{'pca__n_components': 400},0.924210,0.919446,0.915981,0.914249,0.907319,0.916241,0.005611,3,0.617450,0.600000,0.567797,0.558824,0.483871,0.565588,0.046048,4,0.438095,0.342857,0.319048,0.271429,0.213270,0.316940,0.075065,46,0.877168,0.871692,0.867259,0.829148,0.803699,0.849793,0.028583,3,0.321606,0.265480,0.243086,0.217943,0.175088,0.244641,0.048807,2,0.512535,0.436364,0.408537,0.365385,0.296053,0.403774,0.072099,2
4,4,19.806055,0.263003,0.714174,0.035113,500,{'pca__n_components': 500},0.923777,0.913382,0.914682,0.911650,0.906453,0.913989,0.005637,5,0.607595,0.539683,0.555556,0.525424,0.474227,0.540497,0.043240,5,0.457143,0.323810,0.309524,0.295238,0.218009,0.320745,0.077388,45,0.870625,0.866222,0.861245,0.822972,0.796405,0.843494,0.029004,4,0.327130,0.236253,0.234755,0.219222,0.174845,0.238441,0.049594,4,0.521739,0.404762,0.397554,0.378049,0.298701,0.400161,0.071543,3
5,5,17.043679,0.422880,0.920324,0.132935,600,{'pca__n_components': 600},0.917713,0.908185,0.908185,0.906020,0.901256,0.908272,0.005356,6,0.558824,0.493590,0.492063,0.470085,0.417476,0.486408,0.045488,6,0.452381,0.366667,0.295238,0.261905,0.203791,0.315996,0.086157,47,0.863497,0.859804,0.854321,0.797455,0.788280,0.832671,0.032760,6,0.302606,0.238584,0.209373,0.190246,0.157837,0.219729,0.049054,6,0.500000,0.420765,0.369048,0.336391,0.273885,0.380018,0.076608,12
6,6,20.493867,1.488094,0.875779,0.106777,700,{'pca__n_components': 700},0.914249,0.910784,0.904288,0.898224,0.898657,0.905240,0.006409,8,0.53571

In each case, we see the best performance with 300 principal components. We specify the final models and evaluate them on the test set.

In [92]:
best_n_comp=300
logpca_pipe_final=[Pipeline(steps=[('scale', ct_scale[i]),
                             ('pca', PCA(n_components=best_n_comp)),
                            ('logistic', LogisticRegression(max_iter=500, verbose=1, n_jobs=-1))]) for i in [0, 1, 2]]
logpca_scores=[model_score(logpca_pipe_final[i], train_pairs[i][0], train_pairs[i][1], test_pairs[i][0], test_pairs[i][1]) for i in [0, 1, 2]]           

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   10.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    4.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    3.0s finished


In [93]:
logpca_scores

[{'accuracy': 0.9449721091512137,
  'recall': 0.2649164677804296,
  'precision': 0.6607142857142857,
  'auroc': 0.8927869396458686,
  'average precision': 0.4771896143708152,
  'f1': 0.3781942078364566},
 {'accuracy': 0.9394988796088817,
  'recall': 0.2953216374269006,
  'precision': 0.643312101910828,
  'auroc': 0.8786533701599449,
  'average precision': 0.4801284279114553,
  'f1': 0.40480961923847697},
 {'accuracy': 0.9216783216783216,
  'recall': 0.32083333333333336,
  'precision': 0.5579710144927537,
  'auroc': 0.8728435114503816,
  'average precision': 0.5129810402379034,
  'f1': 0.4074074074074075}]

### Logistic regression with regularization

Our other approach to controlling for the high degree of correlation in the input data is to use regularization (both ridge and lasso) to penalize models which assign too high of a coefficient to too many of the covariates. Both approaches add a second term to the loss function which varies proportionately to the magnitude of the regression coefficients (where magnitude is computed via the L1-norm in the case of the lasso and via the L2-norm in the case of the ridge regularization). We cross-validate to find the optimal values of the regularization parameter (C) and method of regularization ('l1' or 'l2').

In [48]:
param_grid={'logistic__C':np.logspace(-3, 1, 5), 'logistic__penalty':['l1', 'l2']}
logistic=LogisticRegression(max_iter=500, verbose=1, solver='saga', n_jobs=-1)
ct_scale=[ColumnTransformer([('scale', StandardScaler(), num_features[i])], remainder="passthrough") for i in [0,1, 2]]
reg_pipe=[Pipeline(steps=[('scale', ct_scale[i]), ('logistic', logistic)]) for i in [0, 1, 2]]
a=time.perf_counter()
logreg_results=[model_eval(reg_pipe[i], train_pairs[i][0], train_pairs[i][1],
                        param_grid=param_grid) for i in [0, 1, 2]]
print(time.perf_counter()-a)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] logistic__C=0.001, logistic__penalty=l1 .........................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.9min finished


[CV]  logistic__C=0.001, logistic__penalty=l1, accuracy=0.937, average_precision=0.066, f1=0.006, precision=1.000, recall=0.003, roc_auc=0.733, total= 3.0min
[CV] logistic__C=0.001, logistic__penalty=l1 .........................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.0min remaining:    0.0s
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.7min finished


[CV]  logistic__C=0.001, logistic__penalty=l1, accuracy=0.937, average_precision=0.069, f1=0.018, precision=0.750, recall=0.009, roc_auc=0.687, total= 2.7min
[CV] logistic__C=0.001, logistic__penalty=l1 .........................


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.7min finished


[CV]  logistic__C=10.0, logistic__penalty=l2, accuracy=0.912, average_precision=0.187, f1=0.302, precision=0.550, recall=0.209, roc_auc=0.821, total= 3.8min


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 198.0min finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


max_iter reached after 423 seconds
31940.434457893003


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  7.1min finished


In [96]:
logreg_results[0]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_logistic__C,param_logistic__penalty,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall,split0_test_roc_auc,split1_test_roc_auc,split2_test_roc_auc,split3_test_roc_auc,split4_test_roc_auc,mean_test_roc_auc,std_test_roc_auc,rank_test_roc_auc,split0_test_average_precision,split1_test_average_precision,split2_test_average_precision,split3_test_average_precision,split4_test_average_precision,mean_test_average_precision,std_test_average_precision,rank_test_average_precision,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1
0,166.359321,6.918668,0.361750,0.111856,0.001,l1,"{'logistic__C': 0.001, 'logistic__penalty': 'l1'}",0.937052,0.937241,0.936864,0.936675,0.936852,0.936937,0.000193,10,1.000000,0.750000,0.000000,0.000000,0.000000,0.350000,0.435890,10,0.002985,0.008955,0.000000,0.000000,0.000000,0.002388,0.003481,10,0.732762,0.686556,0.658325,0.699051,0.676647,0.690668,0.024907,10,0.065933,0.069287,0.063136,0.063325,0.063148,0.064966,0.002406,10,0.005952,0.017699,0.000000,0.000000,0.000000,0.004730,0.006882,10
1,133.585352,2.810770,0.289777,0.026874,0.001,l2,"{'logistic__C': 0.001, 'logistic__penalty': 'l2'}",0.943460,0.942141,0.943649,0.941387,0.939680,0.942063,0.001457,8,0.644628,0.629630,0.725000,0.662338,0.631579,0.658635,0.035178,2,0.232836,0.202985,0.173134,0.151786,0.107463,0.173641,0.042948,8,0.895818,0.867704,0.869014,0.866485,0.861501,0.872104,0.012126,3,0.198528,0.178126,0.177727,0.154246,0.124233,0.166572,0.025392,8,0.342105,0.306998,0.279518,0.246973,0.183673,0.271854,0.054081,8
2,178.825086,1.565729,0.225841,0.036065,0.01,l1,"{'logistic__C': 0.01, 'logistic__penalty': 'l1'}",0.942706,0.943460,0.941764,0.941576,0.939680,0.941837,0.001274,9,0.696203,0.761194,0.732143,0.740741,0.692308,0.724518,0.026479,1,0.164179,0.152239,0.122388,0.119048,0.080597,0.127690,0.029171,9,0.878181,0.843630,0.846809,0.853633,0.832421,0.850935,0.015250,9,0.167072,0.169408,0.145015,0.143969,0.113856,0.147864,0.020064,9,0.265700,0.253731,0.209719,0.205128,0.144385,0.215733,0.042848,9
3,134.065589,1.626754,0.255582,0.055584,0.01,l2,"{'logistic__C': 0.01, 'logistic__penalty': 'l2'}",0.942329,0.943083,0.944779,0.941199,0.940057,0.942289,0.001613,6,0.597315,0.632000,0.718750,0.617647,0.610390,0.635220,0.043243,4,0.265672,0.235821,0.205970,0.187500,0.140299,0.207052,0.042669,6,0.899645,0.871682,0.874088,0.859689,0.857265,0.872474,0.015075,2,0.205052,0.197286,0.198173,0.167260,0.139925,0.181539,0.024548,7,0.367769,0.343478,0.320186,0.287671,0.228155,0.309452,0.048492,6
4,223.891442,2.001418,0.258634,0.027297,0.1,l1,"{'logistic__C': 0.1, 'logistic__penalty': 'l1'}",0.942895,0.943460,0.944591,0.941576,0.940622,0.942629,0.001396,1,0.608108,0.642276,0.725275,0.632653,0.635135,0.648689,0.039985,3,0.268657,0.235821,0.197015,0.184524,0.140299,0.205263,0.043980,7,0.899988,0.872594,0.873865,0.859841,0.859392,0.873136,0.014748,1,0.209546,0.199709,0.193587,0.168379,0.143397,0.182924,0.023992,6,0.372671,0.344978,0.309859,0.285714,0.229829,0.308610,0.049313,7
5,131.651940,2.023167,0.252299,0.034417,0.1,l2,"{'logistic__C': 0.1, 'logistic__penalty': 'l2'}",0.942141,0.943460,0.944591,0.941010,0.940057,0.942252,0.001631,7,0.586420,0.625899,0.711340,0.601770,0.602410,0.625568,0.044704,9,0.283582,0.259701,0.205970,0.202381,0.149254,0.220178,0.047183,5,0.899482,0.871660,0.874097,0.857537,0.855305,0.871616,0.015792,5,0.211530,0.209287,0.196647,0.172296,0.143635,0.186679,0.025643,5,0.382294,0.367089,0.319444,0.302895,0.23923

In [97]:
logreg_results[1]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_logistic__C,param_logistic__penalty,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall,split0_test_roc_auc,split1_test_roc_auc,split2_test_roc_auc,split3_test_roc_auc,split4_test_roc_auc,mean_test_roc_auc,std_test_roc_auc,rank_test_roc_auc,split0_test_average_precision,split1_test_average_precision,split2_test_average_precision,split3_test_average_precision,split4_test_average_precision,mean_test_average_precision,std_test_average_precision,rank_test_average_precision,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1
0,195.157275,51.023795,0.446430,0.115736,0.001,l1,"{'logistic__C': 0.001, 'logistic__penalty': 'l1'}",0.928990,0.928990,0.928990,0.928735,0.928971,0.928935,0.000100,10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10,0.500000,0.667523,0.642970,0.673208,0.653682,0.627477,0.064610,10,0.071010,0.071010,0.071010,0.071265,0.071029,0.071065,0.000100,10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10
1,181.603287,1.046880,0.352243,0.090909,0.001,l2,"{'logistic__C': 0.001, 'logistic__penalty': 'l2'}",0.938152,0.938152,0.935862,0.935353,0.933299,0.936164,0.001836,2,0.647541,0.676471,0.651685,0.696970,0.680851,0.670704,0.018571,2,0.283154,0.247312,0.207885,0.164286,0.114695,0.203466,0.059511,8,0.892758,0.874654,0.884353,0.852138,0.845915,0.869964,0.018137,1,0.234257,0.220748,0.191724,0.174059,0.140972,0.192352,0.033278,8,0.394015,0.362205,0.315217,0.265896,0.196319,0.306730,0.070180,8
2,240.981510,2.342455,0.349320,0.037885,0.01,l1,"{'logistic__C': 0.01, 'logistic__penalty': 'l1'}",0.935607,0.935862,0.935098,0.933825,0.933554,0.934789,0.000935,9,0.685714,0.736842,0.750000,0.763158,0.781250,0.743393,0.032364,1,0.172043,0.150538,0.129032,0.103571,0.089606,0.128958,0.030050,9,0.869265,0.845907,0.845028,0.820827,0.801714,0.836548,0.023197,9,0.176766,0.171243,0.158622,0.142925,0.134668,0.156845,0.016080,9,0.275072,0.250000,0.220183,0.182390,0.160772,0.217683,0.042020,9
3,179.884808,1.196277,0.341373,0.052341,0.01,l2,"{'logistic__C': 0.01, 'logistic__penalty': 'l2'}",0.939934,0.937389,0.934334,0.935862,0.933045,0.936113,0.002403,3,0.659259,0.632000,0.608247,0.662791,0.642857,0.641031,0.019826,4,0.318996,0.283154,0.211470,0.203571,0.129032,0.229245,0.066301,6,0.895443,0.875033,0.886951,0.842749,0.833601,0.866756,0.024392,3,0.258660,0.229857,0.184620,0.191683,0.144813,0.201926,0.039154,6,0.429952,0.391089,0.313830,0.311475,0.214925,0.332254,0.074221,6
4,294.626665,1.076102,0.344969,0.039990,0.1,l1,"{'logistic__C': 0.1, 'logistic__penalty': 'l1'}",0.938916,0.937134,0.936371,0.936625,0.932536,0.936316,0.002089,1,0.640288,0.629032,0.652632,0.701299,0.640000,0.652650,0.025445,3,0.318996,0.279570,0.222222,0.192857,0.114695,0.225668,0.070737,7,0.894995,0.877192,0.886859,0.846646,0.835461,0.868231,0.023166,2,0.252608,0.227017,0.200260,0.192771,0.136287,0.201788,0.038970,7,0.425837,0.387097,0.331551,0.302521,0.194529,0.328307,0.079423,7
5,181.934986,1.479111,0.422117,0.066792,0.1,l2,"{'logistic__C': 0.1, 'logistic__penalty': 'l2'}",0.939679,0.936371,0.934843,0.935862,0.933045,0.935960,0.002180,5,0.650000,0.610687,0.616162,0.655556,0.637931,0.634067,0.017876,6,0.326165,0.286738,0.218638,0.210714,0.132616,0.234974,0.066825,4,0.894775,0.874598,0.886589,0.840677,0.831745,0.865677,0.025060,5,0.259856,0.225756,0.190201,0.194383,0.146209,0.203281,0.037983,4,0.434368,0.390244,0.322751,0.318919,0.2195

In [98]:
logreg_results[2]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_logistic__C,param_logistic__penalty,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall,split0_test_roc_auc,split1_test_roc_auc,split2_test_roc_auc,split3_test_roc_auc,split4_test_roc_auc,mean_test_roc_auc,std_test_roc_auc,rank_test_roc_auc,split0_test_average_precision,split1_test_average_precision,split2_test_average_precision,split3_test_average_precision,split4_test_average_precision,mean_test_average_precision,std_test_average_precision,rank_test_average_precision,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1
0,107.757033,6.698604,0.463888,0.109392,0.001,l1,"{'logistic__C': 0.001, 'logistic__penalty': 'l1'}",0.909052,0.909052,0.909052,0.909052,0.908618,0.908965,0.000173,10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.000000,10,0.090948,0.090948,0.090948,0.090948,0.091382,0.091035,0.000173,10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10
1,206.099107,2.264963,0.433259,0.061575,0.001,l2,"{'logistic__C': 0.001, 'logistic__penalty': 'l2'}",0.923343,0.923777,0.920312,0.918579,0.910784,0.919359,0.004700,7,0.648649,0.684783,0.654762,0.696429,0.545455,0.646015,0.053367,3,0.342857,0.300000,0.261905,0.185714,0.142180,0.246531,0.073433,8,0.889880,0.882597,0.873271,0.847814,0.830920,0.864896,0.022153,2,0.282160,0.269099,0.238614,0.203395,0.155942,0.229842,0.045860,8,0.448598,0.417219,0.374150,0.293233,0.225564,0.351753,0.081871,8
2,263.813537,2.366414,0.573144,0.112417,0.01,l1,"{'logistic__C': 0.01, 'logistic__penalty': 'l1'}",0.920312,0.915115,0.920312,0.913382,0.914249,0.916674,0.003020,9,0.809524,0.705882,0.861111,0.708333,0.760000,0.768970,0.059786,1,0.161905,0.114286,0.147619,0.080952,0.090047,0.118962,0.031521,9,0.843281,0.815883,0.826577,0.806649,0.769641,0.812406,0.024610,9,0.207289,0.161227,0.204639,0.140927,0.151589,0.173134,0.027577,9,0.269841,0.196721,0.252033,0.145299,0.161017,0.204982,0.048956,9
3,207.810606,5.305438,0.406047,0.051773,0.01,l2,"{'logistic__C': 0.01, 'logistic__penalty': 'l2'}",0.926375,0.921178,0.919879,0.919013,0.913382,0.919965,0.004167,2,0.656250,0.627273,0.626263,0.666667,0.573333,0.629957,0.032453,4,0.400000,0.328571,0.295238,0.219048,0.203791,0.289330,0.072221,6,0.889015,0.885819,0.871660,0.839674,0.822207,0.861675,0.026351,3,0.317069,0.267169,0.248994,0.217058,0.189599,0.247978,0.043605,5,0.497041,0.431250,0.401294,0.329749,0.300699,0.392007,0.070529,5
4,326.112890,3.740190,0.570161,0.098078,0.1,l1,"{'logistic__C': 0.1, 'logistic__penalty': 'l1'}",0.930706,0.922477,0.920312,0.918579,0.913382,0.921091,0.005669,1,0.719298,0.647619,0.641304,0.677419,0.593220,0.655772,0.041696,2,0.390476,0.323810,0.280952,0.200000,0.165877,0.272223,0.081544,7,0.894097,0.885340,0.877293,0.849538,0.823199,0.865893,0.026053,1,0.336304,0.271204,0.245572,0.208243,0.174625,0.247190,0.055323,6,0.506173,0.431746,0.390728,0.308824,0.259259,0.379346,0.087584,7
5,209.042703,2.996115,0.541153,0.042731,0.1,l2,"{'logistic__C': 0.1, 'logistic__penalty': 'l2'}",0.927674,0.919879,0.919446,0.920745,0.912083,0.919965,0.004949,2,0.666667,0.610619,0.617647,0.680000,0.550000,0.624987,0.046145,5,0.409524,0.328571,0.300000,0.242857,0.208531,0.297897,0.069888,1,0.888078,0.885442,0.870948,0.837158,0.820630,0.860451,0.026930,6,0.326719,0.261698,0.248958,0.234004,0.187018,0.251679,0.045238,1,0.507375,0.427245,0.403846,0.357895,0.30240

We see that the best regularization parameters for optimizing average precision are the following:

- 12 hour: l1 regularization, C=1.0

- 24 hour: l2 regularization, C=1.0

- 48 hour: l2 regularization, C=0.1

We specify final models with these parameters, train them, and evaluate them on the test set.

In [101]:
ct_scale=[ColumnTransformer([('scale', StandardScaler(), num_features[i])], remainder="passthrough") for i in [0,1, 2]]
logistic12=LogisticRegression(max_iter=500, verbose=1, solver='saga', n_jobs=-1, penalty='l1', C=1.0)
logistic24=LogisticRegression(max_iter=500, verbose=1, solver='saga', n_jobs=-1, penalty='l2', C=1.0)
logistic48=LogisticRegression(max_iter=500, verbose=1, solver='saga', n_jobs=-1, penalty='l2', C=0.1)
reg_pipe_final=[Pipeline(steps=[('scale', ct_scale[0]), ('logistic', logistic12)]),
         Pipeline(steps=[('scale', ct_scale[1]), ('logistic', logistic24)]),
         Pipeline(steps=[('scale', ct_scale[2]), ('logistic', logistic48)])]





In [103]:
logreg_scores=[model_score(reg_pipe_final[i], train_pairs[i][0], train_pairs[i][1], test_pairs[i][0], test_pairs[i][1]) for i in [0, 1, 2]]  



[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


max_iter reached after 282 seconds


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


max_iter reached after 280 seconds


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  4.7min finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


max_iter reached after 238 seconds


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  4.0min finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


max_iter reached after 253 seconds


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  4.2min finished


In [104]:
logreg_scores

[{'accuracy': 0.9442183024272577,
  'recall': 0.24343675417661098,
  'precision': 0.6580645161290323,
  'auroc': 0.8925303783204144,
  'average precision': 0.4696115268233418,
  'f1': 0.3554006968641115},
 {'accuracy': 0.9390914646567529,
  'recall': 0.2543859649122807,
  'precision': 0.6641221374045801,
  'auroc': 0.8793499514057752,
  'average precision': 0.473236165258089,
  'f1': 0.36786469344608885},
 {'accuracy': 0.9230769230769231,
  'recall': 0.32083333333333336,
  'precision': 0.5746268656716418,
  'auroc': 0.8701956106870229,
  'average precision': 0.5099522277094526,
  'f1': 0.41176470588235303}]

In the case of both PCA preprocessing and regularization, we find a substantial improvement in both AUROC and AUPR scores over the baseline logistic regression model.

## Random Forests

The next approach we investigate is random forests. The main parameter we tune is the number of estimators, which is the number of trees in the forest that we train on subsets of the data and average to make our predictions.

In [113]:
ids_sample=train_test_split(train_pairs[0][0].index.get_level_values('hadm_id'), train_size=0.2,stratify=train_pairs[0][1])[0]


In [114]:
train_pairs_sample=[(X[np.isin(X.index.get_level_values('hadm_id'),
                               ids_sample)], y[np.isin(X.index.get_level_values('hadm_id'), ids_sample)])
                       for X, y in train_pairs]

In [116]:
with parallel_backend('threading', n_jobs=4):
    rfc=RandomForestClassifier(verbose=1, n_jobs=-1)
    rf_grid = {'rfc__n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]}
    ct_scale=[ColumnTransformer([('scale', StandardScaler(), num_features[i])], remainder="passthrough") for i in [0,1, 2]]

    rfc_pipe=[Pipeline(steps=[('scale', ct_scale[i]),('rfc', rfc)]) for i in [0, 1, 2]]
    a=time.perf_counter()
    rfc_results=[model_eval(rfc_pipe[i], train_pairs_sample[i][0], train_pairs_sample[i][1],
                        param_grid=rf_grid) for i in [0, 1, 2]]
    print(time.perf_counter()-a)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] rfc__n_estimators=200 ...........................................[CV] rfc__n_estimators=200 ...........................................[CV] rfc__n_estimators=200 ...........................................[CV] rfc__n_estimators=200 ...........................................





[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   15.9s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:   16.3s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_

[CV]  rfc__n_estimators=200, accuracy=0.942, average_precision=0.153, f1=0.244, precision=0.667, recall=0.149, roc_auc=0.839, total=  21.8s
[CV] rfc__n_estimators=200 ...........................................


[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    0.5s finished
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    0.5s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    0.5s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    0.4s finished


[CV]  rfc__n_estimators=200, accuracy=0.938, average_precision=0.106, f1=0.154, precision=0.545, recall=0.090, roc_auc=0.839, total=  22.5s
[CV] rfc__n_estimators=400 ...........................................


...

[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:  1.3min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 1792 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 2000 out of 2000 | elapsed:    0.4s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4

[CV]  rfc__n_estimators=2000, accuracy=0.916, average_precision=0.137, f1=0.133, precision=0.750, recall=0.073, roc_auc=0.706, total= 1.5min


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   10.4s


3897.329379075003


[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:   18.7s
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:   18.8s finished


In [118]:
rfc_results[0]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_rfc__n_estimators,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall,split0_test_roc_auc,split1_test_roc_auc,split2_test_roc_auc,split3_test_roc_auc,split4_test_roc_auc,mean_test_roc_auc,std_test_roc_auc,rank_test_roc_auc,split0_test_average_precision,split1_test_average_precision,split2_test_average_precision,split3_test_average_precision,split4_test_average_precision,mean_test_average_precision,std_test_average_precision,rank_test_average_precision,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1
0,20.946681,1.767685,1.892064,1.088374,200,{'rfc__n_estimators': 200},0.938737,0.941565,0.937795,0.943450,0.939680,0.940245,0.002030,7,0.75,0.666667,0.545455,0.769231,0.636364,0.673543,0.081041,8,0.044776,0.149254,0.089552,0.149254,0.104478,0.107463,0.039376,8,0.884479,0.838794,0.838997,0.839635,0.840393,0.848459,0.018018,9,0.093903,0.153225,0.106340,0.168533,0.123036,0.129007,0.028038,9,0.084507,0.243902,0.153846,0.250000,0.179487,0.182349,0.061252,9
1,44.793174,1.462251,1.040402,0.443597,400,{'rfc__n_estimators': 400},0.938737,0.942507,0.934025,0.942507,0.940622,0.939680,0.003154,10,0.75,0.714286,0.384615,0.750000,0.750000,0.669780,0.143252,10,0.044776,0.149254,0.074627,0.134328,0.089552,0.098507,0.038460,10,0.888255,0.846549,0.838629,0.832014,0.828906,0.846871,0.021560,10,0.093903,0.160333,0.087138,0.155412,0.124657,0.124288,0.030237,10,0.084507,0.246914,0.125000,0.227848,0.160000,0.168854,0.061138,10
2,63.512270,3.937126,1.072650,0.169935,600,{'rfc__n_estimators': 600},0.938737,0.944392,0.936852,0.944392,0.940622,0.940999,0.003016,2,0.75,0.785714,0.500000,0.785714,0.666667,0.697619,0.107960,4,0.044776,0.164179,0.104478,0.164179,0.119403,0.119403,0.044276,1,0.889989,0.840040,0.838494,0.846384,0.834927,0.849967,0.020352,7,0.093903,0.181778,0.108789,0.181778,0.135210,0.140292,0.036366,2,0.084507,0.271605,0.172840,0.271605,0.202532,0.200618,0.069764,1
3,92.730203,2.585958,1.244482,0.092006,800,{'rfc__n_estimators': 800},0.938737,0.943450,0.938737,0.946277,0.939680,0.941376,0.003004,1,0.75,0.769231,0.600000,0.916667,0.636364,0.734452,0.111671,1,0.044776,0.149254,0.089552,0.164179,0.104478,0.110448,0.042844,5,0.891010,0.840889,0.837630,0.834409,0.845731,0.849934,0.020876,8,0.093903,0.168533,0.111224,0.203278,0.123036,0.139995,0.040147,3,0.084507,0.250000,0.155844,0.278481,0.179487,0.189664,0.069042,4
4,96.211450,3.005088,1.573281,0.234840,1000,{'rfc__n_estimators': 1000},0.938737,0.941565,0.936852,0.946277,0.939680,0.940622,0.003210,4,0.75,0.692308,0.500000,0.916667,0.636364,0.699068,0.136784,3,0.044776,0.134328,0.104478,0.164179,0.104478,0.110448,0.039601,5,0.884696,0.844432,0.841639,0.842465,0.841782,0.851003,0.016876,5,0.093903,0.147662,0.108789,0.203278,0.123036,0.135334,0.038317,5,0.084507,0.225000,0.172840,0.278481,0.179487,0.188063,0.064139,5
5,104.482486,2.102418,1.666285,0.066260,1200,{'rfc__n_estimators': 1200},0.938737,0.941565,0.936852,0.944392,0.939680,0.940245,0.002571,7,0.75,0.666667,0.500000,0.833333,0.636364,0.677273,0.112121,7,0.044776,0.149254,0.089552,0.149254,0.104478,0.107463,0.039376,8,0.886558,0.846437,0.840213,0.839665,0.842908,0.851156,0.017863,4,0.093903,0.153225,0.102269,0.178101,0.123036,0.130107,0.031539,8,0.084507,0.243902,0.151899,0.253165,0.179487,0.182592,0.062148,8
6,119.919428,1.741140,2.030294,0.149198,1400,{'rfc__n_estimators': 1400},0.938737,0.942507,0.934967,0.944392,0.941565,0.940434,0.003287,5,0.75,0.687500,0.416667,0.833333,0.727273,0.682955,0

In [119]:
rfc_results[1]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_rfc__n_estimators,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall,split0_test_roc_auc,split1_test_roc_auc,split2_test_roc_auc,split3_test_roc_auc,split4_test_roc_auc,mean_test_roc_auc,std_test_roc_auc,rank_test_roc_auc,split0_test_average_precision,split1_test_average_precision,split2_test_average_precision,split3_test_average_precision,split4_test_average_precision,mean_test_average_precision,std_test_average_precision,rank_test_average_precision,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1
0,20.637769,1.147793,0.827100,0.165321,200,{'rfc__n_estimators': 200},0.934699,0.935980,0.933419,0.937179,0.933333,0.934922,0.001486,4,1.0,0.750000,0.750000,0.818182,0.833333,0.830303,0.091488,2,0.089286,0.160714,0.107143,0.160714,0.089286,0.121429,0.032733,5,0.869963,0.819495,0.826084,0.796752,0.808948,0.824248,0.024924,9,0.154587,0.180715,0.144378,0.191750,0.139789,0.162244,0.020464,4,0.163934,0.264706,0.187500,0.268657,0.161290,0.209217,0.047815,5
1,38.473063,1.547880,0.867961,0.209822,400,{'rfc__n_estimators': 400},0.934699,0.935980,0.928297,0.938462,0.934615,0.934411,0.003358,5,1.0,0.750000,0.500000,0.833333,0.857143,0.788095,0.165026,5,0.089286,0.160714,0.071429,0.178571,0.107143,0.121429,0.041342,7,0.858916,0.815751,0.830012,0.800205,0.818407,0.824658,0.019589,8,0.154587,0.180715,0.102296,0.207784,0.155939,0.160264,0.034893,7,0.163934,0.264706,0.125000,0.294118,0.190476,0.207647,0.062879,7
2,60.418086,1.919908,0.872860,0.054107,600,{'rfc__n_estimators': 600},0.934699,0.934699,0.930858,0.939744,0.934615,0.934923,0.002827,1,1.0,0.727273,0.625000,0.846154,1.000000,0.839685,0.148438,1,0.089286,0.142857,0.089286,0.196429,0.089286,0.121429,0.042857,5,0.863387,0.823805,0.823732,0.797812,0.812031,0.824153,0.021819,10,0.154587,0.165356,0.121104,0.223901,0.154670,0.163924,0.033483,3,0.163934,0.238806,0.156250,0.318841,0.163934,0.208353,0.062922,6
3,74.031684,2.437164,0.985687,0.034421,800,{'rfc__n_estimators': 800},0.932138,0.935980,0.929577,0.938462,0.932051,0.933642,0.003162,10,1.0,0.750000,0.555556,0.833333,0.714286,0.770635,0.145876,10,0.053571,0.160714,0.089286,0.178571,0.089286,0.114286,0.047380,9,0.867229,0.819261,0.830702,0.796653,0.814683,0.825706,0.023479,7,0.121433,0.180715,0.114904,0.207784,0.129160,0.150799,0.036773,10,0.101695,0.264706,0.153846,0.294118,0.158730,0.194619,0.072660,10
4,94.712193,2.345024,1.184073,0.018522,1000,{'rfc__n_estimators': 1000},0.933419,0.935980,0.929577,0.938462,0.932051,0.933898,0.003081,9,1.0,0.750000,0.571429,0.833333,0.714286,0.773810,0.141261,9,0.071429,0.160714,0.071429,0.178571,0.089286,0.114286,0.046015,9,0.863707,0.823251,0.829631,0.802412,0.821429,0.828086,0.019990,3,0.138010,0.180715,0.107398,0.207784,0.129160,0.152613,0.036445,9,0.133333,0.264706,0.126984,0.294118,0.158730,0.195574,0.069894,9
5,118.671910,11.437414,1.598452,0.273104,1200,{'rfc__n_estimators': 1200},0.934699,0.935980,0.929577,0.938462,0.933333,0.934410,0.002949,7,1.0,0.750000,0.555556,0.833333,0.750000,0.777778,0.143802,8,0.089286,0.160714,0.089286,0.178571,0.107143,0.125000,0.037457,2,0.863842,0.822685,0.829520,0.808924,0.820343,0.829063,0.018612,1,0.154587,0.180715,0.114904,0.207784,0.144460,0.160490,0.031672,5,0.163934,0.264706,0.153846,0.294118,0.187500,0.212821,0.056232,3
6,167.134494,2.456300,2.289859,0.233975,1400,{'rfc__n_estimators': 1400},0.934699,0.935980,0.930858,0.939744,0.933333,0.934923,0.002948,3,1.0,0.750000,0.625000,0.846154,0.750000,0.794231,0.124570

In [120]:
rfc_results[2]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_rfc__n_estimators,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,split4_test_accuracy,mean_test_accuracy,std_test_accuracy,rank_test_accuracy,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,mean_test_precision,std_test_precision,rank_test_precision,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,split4_test_recall,mean_test_recall,std_test_recall,rank_test_recall,split0_test_roc_auc,split1_test_roc_auc,split2_test_roc_auc,split3_test_roc_auc,split4_test_roc_auc,mean_test_roc_auc,std_test_roc_auc,rank_test_roc_auc,split0_test_average_precision,split1_test_average_precision,split2_test_average_precision,split3_test_average_precision,split4_test_average_precision,mean_test_average_precision,std_test_average_precision,rank_test_average_precision,split0_test_f1,split1_test_f1,split2_test_f1,split3_test_f1,split4_test_f1,mean_test_f1,std_test_f1,rank_test_f1
0,17.625933,1.123125,1.074646,0.198217,200,{'rfc__n_estimators': 200},0.926724,0.915948,0.924406,0.924406,0.917927,0.921882,0.004172,10,1.0,0.800000,0.800000,1.0,0.80,0.880000,0.097980,9,0.190476,0.095238,0.195122,0.146341,0.097561,0.144948,0.043148,10,0.838439,0.749323,0.807248,0.757542,0.684227,0.767356,0.052870,10,0.263752,0.158087,0.227372,0.221935,0.157962,0.205822,0.041591,10,0.320000,0.170213,0.313725,0.255319,0.173913,0.246634,0.064943,10
1,31.458578,2.435551,1.028660,0.208706,400,{'rfc__n_estimators': 400},0.928879,0.920259,0.924406,0.924406,0.917927,0.923175,0.003785,1,1.0,0.857143,0.800000,1.0,0.80,0.891429,0.091071,8,0.214286,0.142857,0.195122,0.146341,0.097561,0.159233,0.041368,1,0.827014,0.766390,0.798809,0.776933,0.697607,0.773351,0.043189,9,0.285406,0.200035,0.227372,0.221935,0.157962,0.218542,0.041407,1,0.352941,0.244898,0.313725,0.255319,0.173913,0.268159,0.061404,1
2,43.971762,1.997497,1.032609,0.099333,600,{'rfc__n_estimators': 600},0.926724,0.920259,0.924406,0.924406,0.917927,0.922744,0.003186,2,1.0,1.000000,0.800000,1.0,0.80,0.920000,0.097980,1,0.190476,0.119048,0.195122,0.146341,0.097561,0.149710,0.038458,4,0.833841,0.762695,0.794157,0.775199,0.713646,0.775908,0.039347,5,0.263752,0.198789,0.227372,0.221935,0.157962,0.213962,0.034896,3,0.320000,0.212766,0.313725,0.255319,0.173913,0.255145,0.056626,4
3,59.337862,2.748197,1.130078,0.035337,800,{'rfc__n_estimators': 800},0.926724,0.920259,0.922246,0.924406,0.917927,0.922312,0.003076,6,1.0,0.857143,0.727273,1.0,0.80,0.876883,0.108627,10,0.190476,0.142857,0.195122,0.146341,0.097561,0.154472,0.035745,2,0.841345,0.769634,0.808519,0.775951,0.721131,0.783316,0.040277,1,0.263752,0.200035,0.213181,0.221935,0.157962,0.211373,0.034175,6,0.320000,0.244898,0.307692,0.255319,0.173913,0.260364,0.052029,3
4,78.925289,2.540388,1.380225,0.202908,1000,{'rfc__n_estimators': 1000},0.926724,0.920259,0.924406,0.924406,0.917927,0.922744,0.003186,2,1.0,1.000000,0.800000,1.0,0.80,0.920000,0.097980,1,0.190476,0.119048,0.195122,0.146341,0.097561,0.149710,0.038458,4,0.834744,0.757532,0.803867,0.778349,0.705930,0.776084,0.043552,3,0.263752,0.198789,0.227372,0.221935,0.157962,0.213962,0.034896,3,0.320000,0.212766,0.313725,0.255319,0.173913,0.255145,0.056626,4
5,91.959392,4.391158,1.523642,0.141852,1200,{'rfc__n_estimators': 1200},0.926724,0.922414,0.922246,0.924406,0.917927,0.922743,0.002904,5,1.0,1.000000,0.727273,1.0,0.80,0.905455,0.118056,4,0.190476,0.142857,0.195122,0.146341,0.097561,0.154472,0.035745,2,0.837311,0.765064,0.797567,0.774477,0.705785,0.776041,0.043072,4,0.263752,0.220443,0.213181,0.221935,0.157962,0.215455,0.033794,2,0.320000,0.250000,0.307692,0.255319,0.173913,0.261385,0.051765,2
6,116.002615,5.644661,1.780907,0.188253,1400,{'rfc__n_estimators': 1400},0.926724,0.920259,0.922246,0.924406,0.917927,0.922312,0.003076,6,1.0,1.000000,0.727273,1.0,0.80,0.905455,0.118056,4,0.190476,0.119048,0.195122,0.146341,0.097561,0.149710,0.03845

In [121]:
best_params=[1800, 1400, 400]
ct_scale=[ColumnTransformer([('scale', StandardScaler(), num_features[i])], remainder="passthrough") for i in [0,1, 2]]
rfc=[RandomForestClassifier(verbose=1, n_jobs=-1, n_estimators=best_params[i]) for i in [0, 1, 2]]
rfc_pipe_final=[Pipeline(steps=[('scale', ct_scale[i]),('rfc', rfc[i])]) for i in [0, 1, 2]]

In [122]:
rfc_scores=[model_score(rfc_pipe_final[i], train_pairs[i][0], train_pairs[i][1], test_pairs[i][0], test_pairs[i][1]) for i in [0, 1, 2]]  

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   29.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 1800 out of 1800 | elapsed:  7.6min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    1.3s
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed:    1.8s
[Parallel(n_jobs=4)]: Done 1792 tasks      | elapsed:    2.5s
[Parallel(n_jobs=4)]: Do

In [124]:
rfc_scores

[{'accuracy': 0.9434644957033017,
  'recall': 0.14558472553699284,
  'precision': 0.782051282051282,
  'auroc': 0.8920210964079109,
  'average precision': 0.4931000508546054,
  'f1': 0.2454728370221328},
 {'accuracy': 0.9386840497046242,
  'recall': 0.14912280701754385,
  'precision': 0.8360655737704918,
  'auroc': 0.8837125475538348,
  'average precision': 0.501491070014298,
  'f1': 0.2531017369727047},
 {'accuracy': 0.929020979020979,
  'recall': 0.17916666666666667,
  'precision': 0.8775510204081632,
  'auroc': 0.8763907442748091,
  'average precision': 0.5047308979358249,
  'f1': 0.2975778546712803}]

## Support vector machines

In the case of support vector classifiers, the hyperparameters that must be set are the kernel that we use (linear, polynomial, radial basis functions, or sigmoid), the kernel coefficient gamma, and the regularization coeficient C. Because of the computational resources required for tuning the parameters, in this case we use sklearn's Halving Random Search to tune hyperparameters, which uses only a fraction of the data to test all parameter combinations, then eliminating parameters and running the remaining combinations on more and more of the data. While less thorough than GridSearch, this runs more quickly.

In [231]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

In [232]:
max_resources=[np.floor(train_pairs[i][0].shape[0]*0.7).astype(int) for i in [0, 1, 2]]
min_resources=[np.floor(max_resources[i]/5).astype(int) for i in [0, 1, 2]]

In [87]:
ct_scale=[ColumnTransformer([('scale', StandardScaler(), num_features[i])], remainder="passthrough") for i in [0,1, 2]]
X_scaled=[ct_scale[i].fit_transform(train_pairs[i][0]) for i in [0, 1, 2]]

In [234]:
C_range = np.logspace(-2, 1, 5)
gamma_range = np.logspace(-4, -3, 5)
ker=['linear', 'poly', 'rbf', 'sigmoid']
svm_mod=svm.SVC(class_weight='balanced', verbose=-1, probability=True, cache_size=2000)
param_grid = {'gamma':gamma_range, 'C':C_range, 'kernel':ker}
search = [HalvingRandomSearchCV(svm_mod, param_grid,
                                resource='n_samples',
                                max_resources=max_resources[i],
                                min_resources=min_resources[i],
                                random_state=2,
                                cv=3,
                                scoring=make_scorer(roc_auc_score, needs_proba=True),
                                verbose=4,n_jobs=-1) for i in [0, 1, 2]]
with parallel_backend('threading', n_jobs=-1):
    for i in [0, 1, 2]:
        #print("Searching parameters for model ", i)
        search[i].fit(X_scaled[i], train_pairs[i][1])

n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 3714
max_resources_: 18570
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 5
n_resources: 3714
Fitting 3 folds for each of 5 candidates, totalling 15 fits
[LibSVM][LibSVM][LibSVM][LibSVM][CV 1/3] END C=10.0, gamma=0.0001, kernel=sigmoid;, score=(train=0.896, test=0.853) total time= 1.1min
[LibSVM][CV 2/3] END C=10.0, gamma=0.0001, kernel=sigmoid;, score=(train=0.900, test=0.779) total time= 1.1min
[LibSVM][CV 3/3] END C=10.0, gamma=0.0001, kernel=sigmoid;, score=(train=0.912, test=0.859) total time= 1.2min
[LibSVM][CV 1/3] END C=0.31622776601683794, gamma=0.001, kernel=linear;, score=(train=0.992, test=0.771) total time=  52.8s
[LibSVM][CV 1/3] END C=0.05623413251903491, gamma=0.00031622776601683794, kernel=rbf;, score=(train=0.881, test=0.852) total time= 2.1min
[LibSVM][CV 2/3] END C=0.31622776601683794, gamma=0.001, kernel=linear;, score=(train=0.992, test=0.670) total time=  4

[LibSVM][LibSVM][LibSVM][LibSVM][CV 3/3] END C=10.0, gamma=0.0001, kernel=sigmoid;, score=(train=0.794, test=0.763) total time=15.5min
[LibSVM][CV 2/3] END C=10.0, gamma=0.0001, kernel=sigmoid;, score=(train=0.788, test=0.775) total time=16.7min
[LibSVM][CV 1/3] END C=10.0, gamma=0.0001, kernel=sigmoid;, score=(train=0.753, test=0.783) total time=19.9min
[CV 1/3] END C=0.05623413251903491, gamma=0.00031622776601683794, kernel=rbf;, score=(train=0.859, test=0.797) total time=33.1min
[CV 2/3] END C=0.05623413251903491, gamma=0.00031622776601683794, kernel=rbf;, score=(train=0.872, test=0.782) total time=21.2min
[CV 3/3] END C=0.05623413251903491, gamma=0.00031622776601683794, kernel=rbf;, score=(train=0.883, test=0.794) total time=19.9min
[LibSVM]

In each case, we find the best results with a radial basis function kernel with gamma and C parameters as given below:

In [239]:
for i in [0,1,2]:
    print(search[i].best_params_)

{'kernel': 'rbf', 'gamma': 0.00031622776601683794, 'C': 0.05623413251903491}
{'kernel': 'rbf', 'gamma': 0.00031622776601683794, 'C': 0.05623413251903491}
{'kernel': 'rbf', 'gamma': 0.00031622776601683794, 'C': 0.05623413251903491}


In [4]:
svm_final=svm.SVC(class_weight='balanced', probability=True, cache_size=2000, 
                     kernel='rbf', gamma=0.00031622776601683794, C=0.05623413251903491, verbose=4)
ct_scale=[ColumnTransformer([('scale', StandardScaler(),
                              num_features[i])], remainder="passthrough") for i in [0,1, 2]]
#svm_final=[svm.SVC(class_weight='balanced', verbose=1, probability=True, kernel=ker[i]) for i in [0, 1, 2]] 
svm_pipe_final=[Pipeline(steps=[('scale', ct_scale[i]),('svm', svm_final)]) for i in [0, 1, 2]]

In [5]:
svm_scores=[model_score(svm_pipe_final[i], train_pairs[i][0], train_pairs[i][1], test_pairs[i][0], test_pairs[i][1]) for i in [0, 1, 2]]  

[LibSVM][LibSVM][LibSVM]

In [6]:
svm_scores

[{'accuracy': 0.7684305744007237,
  'recall': 0.837708830548926,
  'precision': 0.19296316657504123,
  'auroc': 0.8817294537778656,
  'average precision': 0.3880918895454234,
  'f1': 0.3136729222520107},
 {'accuracy': 0.7421063353025056,
  'recall': 0.8391812865497076,
  'precision': 0.19158878504672897,
  'auroc': 0.8675749753187435,
  'average precision': 0.3638418392791448,
  'f1': 0.31195652173913047},
 {'accuracy': 0.6888111888111889,
  'recall': 0.825,
  'precision': 0.18929254302103252,
  'auroc': 0.8280804707379136,
  'average precision': 0.3361115671567237,
  'f1': 0.30793157076205285}]

## Deep learning models

### Preprocessing for deep learning models

We implement our deep learning models in the Keras framework, and as a result we must do some additional processing (such as normalizing the data) prior to specifying and running the models. We also now begin working with the chart note data, so we encode this using TF-IDF. Finally, we tuned each model dozens of times with various architectures (number and size of layers, degree of dropout). To do this tuning, we split out training set further to obtain a validation set on which we evaluate our models. We do this split for both the numeric and chart note data.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidforize(Xtrain, Xtest):
    vectorizer=TfidfVectorizer()
    X_train_tfidf=vectorizer.fit_transform(Xtrain)
    X_test_tfidf=vectorizer.transform(Xtest)
    X_train_tfidf.sort_indices()
    X_test_tfidf.sort_indices()
    return X_train_tfidf, X_test_tfidf

ct_scale=[ColumnTransformer([('scale', StandardScaler(), num_features[i])], remainder="passthrough") for i in [0,1, 2]]
X_scaled=[ct_scale[i].fit_transform(train_pairs[i][0]) for i in [0, 1, 2]]

X_combined2=[pd.concat([pd.DataFrame(X_scaled[i]), pd.DataFrame(text_train_pairs[i][0].values)], axis=1) for i in [0, 1, 2]]
combined_splits=[train_test_split(
        X_combined2[i], train_pairs[i][1],
        test_size=0.2, stratify=train_pairs[i][1],
        random_state=33) for i in [0, 1, 2]]


X_train_split_combined=[combined_splits[i][0] for i in [0, 1, 2]]
X_val_split_combined=[combined_splits[i][1] for i in [0, 1, 2]]
y_train_split=[combined_splits[i][2] for i in [0, 1, 2]]
y_val_split=[combined_splits[i][3] for i in [0, 1, 2]]

X_train_split_num=[X_train_split_combined[i].iloc[:, 0:X_scaled[i].shape[1]] for i in [0, 1, 2]]
X_train_split_text=[X_train_split_combined[i].iloc[:, X_scaled[i].shape[1]:] for i in [0, 1, 2]]
X_val_split_num=[X_val_split_combined[i].iloc[:, 0:X_scaled[i].shape[1]] for i in [0, 1, 2]]
X_val_split_text=[X_val_split_combined[i].iloc[:, X_scaled[i].shape[1]:] for i in [0, 1, 2]]

for i in [0, 1, 2]:
    X_train_split_text[i].columns=["TEXT"]
    X_val_split_text[i].columns=["TEXT"]

tfidf_pairs=[tfidforize(X_train_split_text[i]["TEXT"], X_val_split_text[i]["TEXT"]) for i in [0, 1, 2]]
X_train_split_tfidf=[tfidf_pairs[i][0] for i in [0, 1, 2]]
X_val_split_tfidf=[tfidf_pairs[i][1] for i in [0, 1, 2]]

In [101]:
def score_nn(trained_model, test_scaled, y_test):
    y_prob=trained_model.predict(test_scaled)
    y_pred=y_prob.copy()
    y_pred[y_prob<0.5]=0
    y_pred[y_prob>=0.5]=1
    cc=accuracy_score(y_test, y_pred)
    rec=recall_score(y_test, y_pred)
    prec=precision_score(y_test, y_pred)
    auroc=roc_auc_score(y_test, y_prob)
    ap=average_precision_score(y_test, y_prob)
    f1=f1_score(y_test, y_pred)
    score={'accuracy':acc, 'recall':rec, 'precision':prec, 'auroc':auroc, 'average precision':ap, 'f1':f1}
    return score

### Multilayer perceptron

We begin by implementing a multilayer perceptron model, which we run for each of the three window sizes.

In [99]:
def nn_model(i):
    nn_model = Sequential()
    nn_model.add(Dense(512, input_dim=X_scaled[i].shape[1], activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.005)))
    nn_model.add(Dropout(0.5))
    nn_model.add(Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.005)))
    nn_model.add(Dropout(0.3))
    nn_model.add(Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.005)))
    nn_model.add(Dense(1, activation='sigmoid'))
    
    opt = keras.optimizers.Adam(learning_rate=0.00005)
    nn_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[keras.metrics.AUC(), keras.metrics.Recall(), keras.metrics.Precision(), keras.metrics.AUC(name='prc', curve='PR')])
    print(nn_model.summary())
    return nn_model

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)


In [97]:
m1 = nn_model(0)

m1.fit(X_train_split_num[0], y_train_split[0], validation_data=(X_val_split_num[0], y_val_split[0]), callbacks=[callback],
      epochs=50, batch_size=30, verbose=1)

In [94]:
m1_final=nn_model(0)
m1_final.fit(X_scaled[0], text_train_pairs[0][1], epochs=27, batch_size=30, verbose=1)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_83 (Dense)             (None, 512)               719872    
_________________________________________________________________
dropout_30 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_84 (Dense)             (None, 128)               65664     
_________________________________________________________________
dropout_31 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_85 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_86 (Dense)             (None, 1)                 65        
Total params: 793,857
Trainable params: 793,857
Non-trainable params: 0
________________________________________________

/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


  1/222 [..............................] - ETA: 4s

/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


222/222 [==============================] - 1s 2ms/step


In [95]:
score_nn(m1_final, X_scaled_test[0], text_test_pairs[0][1])

{'accuracy': 0.9455751545303784,
 'recall': 0.315035799522673,
 'precision': 0.6407766990291263,
 'auroc': 0.9128394348583881,
 'average precision': 0.5068135755726634,
 'f1': 0.4224}

In [100]:
m2 = nn_model(1)

m2.fit(X_train_split_num[1], y_train_split[1], validation_data=(X_val_split_num[1], y_val_split[1]), callbacks=[callback],
      epochs=50, batch_size=30, verbose=1)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_87 (Dense)             (None, 512)               1358848   
_________________________________________________________________
dropout_32 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_88 (Dense)             (None, 128)               65664     
_________________________________________________________________
dropout_33 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_89 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_90 (Dense)             (None, 1)                 65        
Total params: 1,432,833
Trainable params: 1,432,833
Non-trainable params: 0
___________________________________________

In [102]:
m2_final=nn_model(1)

m2_final.fit(X_scaled[1], text_train_pairs[1][1], epochs=31, batch_size=30, verbose=1)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_91 (Dense)             (None, 512)               1358848   
_________________________________________________________________
dropout_34 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_92 (Dense)             (None, 128)               65664     
_________________________________________________________________
dropout_35 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_93 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_94 (Dense)             (None, 1)                 65        
Total params: 1,432,833
Trainable params: 1,432,833
Non-trainable params: 0
___________________________________________

In [104]:
score_nn(m2_final, X_scaled_test[1], text_test_pairs[1][1])

{'accuracy': 0.9478365747022464,
 'recall': 0.34502923976608185,
 'precision': 0.6742857142857143,
 'auroc': 0.9023774676454657,
 'average precision': 0.5276047155447614,
 'f1': 0.4564796905222437}

We found that for the 48 hour window, the model performed better with less regularization, which we specify here:

In [110]:
def nn_model_b(i):
    nn_model = Sequential()
    nn_model.add(Dense(512, input_dim=X_scaled[i].shape[1], activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.0005)))
    nn_model.add(Dropout(0.5))
    nn_model.add(Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.0005)))
    nn_model.add(Dropout(0.3))
    nn_model.add(Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.0005)))
    nn_model.add(Dense(1, activation='sigmoid'))
    
    opt = keras.optimizers.Adam(learning_rate=0.00005)
    nn_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[keras.metrics.AUC(), keras.metrics.Recall(), keras.metrics.Precision(), keras.metrics.AUC(name='prc', curve='PR')])
    print(nn_model.summary())
    return nn_model

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)


In [111]:
m3 = nn_model_b(2)

m3.fit(X_train_split_num[2], y_train_split[2], validation_data=(X_val_split_num[2], y_val_split[2]), callbacks=[callback],
      epochs=100, batch_size=30, verbose=1)

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_107 (Dense)            (None, 512)               2636800   
_________________________________________________________________
dropout_42 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_108 (Dense)            (None, 128)               65664     
_________________________________________________________________
dropout_43 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_109 (Dense)            (None, 64)                8256      
_________________________________________________________________
dense_110 (Dense)            (None, 1)                 65        
Total params: 2,710,785
Trainable params: 2,710,785
Non-trainable params: 0
___________________________________________

In [112]:
m3_final=nn_model_b(2)

m3_final.fit(X_scaled[2], text_train_pairs[2][1], epochs=23, batch_size=30, verbose=1)

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_111 (Dense)            (None, 512)               2636800   
_________________________________________________________________
dropout_44 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_112 (Dense)            (None, 128)               65664     
_________________________________________________________________
dropout_45 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_113 (Dense)            (None, 64)                8256      
_________________________________________________________________
dense_114 (Dense)            (None, 1)                 65        
Total params: 2,710,785
Trainable params: 2,710,785
Non-trainable params: 0
___________________________________________

In [114]:
score_nn(m3_final, X_scaled_test[2], text_test_pairs[2][1])

{'accuracy': 0.9478365747022464,
 'recall': 0.4041666666666667,
 'precision': 0.5574712643678161,
 'auroc': 0.8636688931297709,
 'average precision': 0.5281346007011438,
 'f1': 0.46859903381642515}

### NN models on chart note data using TF-IDF encoding

To score the chart note models, we write a separate scoring function since they have different input data from the NN models above.

In [ ]:
def score_tfidf(trained_model, X_tfidf, y_test):
    y_prob=trained_model.predict(X_tfidf)
    y_pred=y_prob.copy()
    y_pred[y_prob<0.5]=0
    y_pred[y_prob>=0.5]=1
    acc=accuracy_score(y_test, y_pred)
    rec=recall_score(y_test, y_pred)
    prec=precision_score(y_test, y_pred)
    auroc=roc_auc_score(y_test, y_prob)
    ap=average_precision_score(y_test, y_prob)
    f1=f1_score(y_test, y_pred)
    score={'accuracy':acc, 'recall':rec, 'precision':prec, 'auroc':auroc, 'average precision':ap, 'f1':f1}
    return score

In [74]:
train_test_tfidf=[tfidforize(text_train_pairs[i][0]["TEXT"], text_test_pairs[i][0]["TEXT"]) for i in [0, 1, 2]]

train_tfidf=[train_test_tfidf[i][0] for i in [0, 1, 2]]
test_tfidf=[train_test_tfidf[i][1] for i in [0, 1, 2]]

In [75]:
def model_tfidf(i):
    model_tfidf = Sequential()
    model_tfidf.add(Dense(32, input_dim=train_tfidf[i].shape[1], activation='relu', 
                          kernel_regularizer=tf.keras.regularizers.l2(l2=0.0001)))
    model_tfidf.add(Dense(1, activation='sigmoid'))
    
    opt = keras.optimizers.Adam(learning_rate=0.0001)
    model_tfidf.compile(loss='binary_crossentropy', optimizer=opt, metrics=[keras.metrics.AUC(), keras.metrics.Recall(), keras.metrics.Precision(), keras.metrics.AUC(name='prc', curve='PR')])
    print(model_tfidf.summary())
    return model_tfidf

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
m_tfidf1 = model_tfidf(0)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_65 (Dense)             (None, 32)                4755520   
_________________________________________________________________
dense_66 (Dense)             (None, 1)                 33        
Total params: 4,755,553
Trainable params: 4,755,553
Non-trainable params: 0
_________________________________________________________________
None


In [76]:
splits_tfidf=[train_test_split(
    train_tfidf[i], text_train_pairs[i][1], stratify=text_train_pairs[i][1], test_size=0.2, shuffle=True) for i in [0, 1, 2]]

X_train_tfidf=[splits_tfidf[i][0] for i in [0, 1, 2]]
X_val_tfidf=[splits_tfidf[i][1] for i in [0, 1, 2]]
y_train_tfidf=[splits_tfidf[i][2] for i in [0, 1, 2]]
y_val_tfidf=[splits_tfidf[i][3] for i in [0, 1, 2]]

for i in [0, 1, 2]:
    X_train_tfidf[i].sort_indices()
    X_val_tfidf[i].sort_indices()


First, we run the model on the 12 hour data.

In [77]:
m_tfidf1.fit(X_train_tfidf[0], y_train_tfidf[0], epochs=30, batch_size=30, verbose=1, validation_data=(X_val_tfidf[0], y_val_tfidf[0]))

Epoch 1/30
708/708 [==============================] - 38s 45ms/step - loss: 0.5520 - auc_15: 0.4850 - recall_15: 0.0145 - precision_15: 0.0638 - prc: 0.0620 - val_loss: 0.3051 - val_auc_15: 0.4033 - val_recall_15: 0.0000e+00 - val_precision_15: 0.0000e+00 - val_prc: 0.0535
Epoch 2/30
708/708 [==============================] - 36s 50ms/step - loss: 0.2892 - auc_15: 0.4422 - recall_15: 0.0000e+00 - precision_15: 0.0000e+00 - prc: 0.0601 - val_loss: 0.2692 - val_auc_15: 0.4413 - val_recall_15: 0.0000e+00 - val_precision_15: 0.0000e+00 - val_prc: 0.0571
Epoch 3/30
708/708 [==============================] - 36s 51ms/step - loss: 0.2661 - auc_15: 0.4887 - recall_15: 0.0000e+00 - precision_15: 0.0000e+00 - prc: 0.0685 - val_loss: 0.2572 - val_auc_15: 0.4966 - val_recall_15: 0.0000e+00 - val_precision_15: 0.0000e+00 - val_prc: 0.0649
Epoch 4/30
708/708 [==============================] - 31s 44ms/step - loss: 0.2470 - auc_15: 0.5640 - recall_15: 0.0000e+00 - precision_15: 0.0000e+00 - prc: 0.07

After tuning parameters, we specify a final model and score it on the test set.

In [78]:
m_tfidf1_final=model_tfidf(0)
m_tfidf1_final.fit(train_tfidf[0], text_train_pairs[0][1], epochs=30, batch_size=30, verbose=1)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_67 (Dense)             (None, 32)                4755520   
_________________________________________________________________
dense_68 (Dense)             (None, 1)                 33        
Total params: 4,755,553
Trainable params: 4,755,553
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/30
885/885 [==============================] - 57s 60ms/step - loss: 0.4817 - auc_16: 0.4680 - recall_16: 0.0105 - precision_16: 0.0418 - prc: 0.0563
Epoch 2/30
885/885 [==============================] - 51s 58ms/step - loss: 0.2634 - auc_16: 0.4773 - recall_16: 0.0000e+00 - precision_16: 0.0000e+00 - prc: 0.0636
Epoch 3/30
885/885 [==============================] - 53s 60ms/step - loss: 0.2480 - auc_16: 0.5626 - recall_16: 0.0000e+00 - precision_16: 0.0000e+00 - prc: 0.0759
Epoch 4/30
885

In [80]:
score_tfidf(m_tfidf1_final, test_tfidf[0], text_test_pairs[0][1])

{'accuracy': 0.9446705864616313,
 'recall': 0.2052505966587112,
 'precision': 0.7166666666666667,
 'auroc': 0.883241360450995,
 'average precision': 0.4520645076146739,
 'f1': 0.31910946196660483}

Next we run the model on the 24 hour data.

In [42]:
m_tfidf2 = model_tfidf(1)
m_tfidf2.fit(X_train_tfidf[1], y_train_tfidf[1], epochs=30, batch_size=30, verbose=1, validation_data=(X_val_tfidf[1], y_val_tfidf[1]))


ValueError: Input 0 of layer sequential_1 is incompatible with the layer: : expected min_ndim=2, found ndim=0. Full shape received: ()

In [373]:
for i in [0, 1, 2]:
    train_tfidf[i].sort_indices()
    test_tfidf[i].sort_indices()

In [374]:
m_tfidf2_final=model_tfidf(1)
m_tfidf2_final.fit(train_tfidf[1], text_train_pairs[1][1], epochs=30, batch_size=30, verbose=1)

Model: "sequential_130"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_251 (Dense)            (None, 32)                4121088   
_________________________________________________________________
dense_252 (Dense)            (None, 1)                 33        
Total params: 4,121,121
Trainable params: 4,121,121
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/30
655/655 [==============================] - 29s 39ms/step - loss: 0.5349 - auc_127: 0.4714 - recall_127: 0.0068 - precision_127: 0.0385 - prc: 0.0651
Epoch 2/30
655/655 [==============================] - 24s 37ms/step - loss: 0.2998 - auc_127: 0.4792 - recall_127: 0.0000e+00 - precision_127: 0.0000e+00 - prc: 0.0858
Epoch 3/30
655/655 [==============================] - 25s 37ms/step - loss: 0.2719 - auc_127: 0.5208 - recall_127: 0.0000e+00 - precision_127: 0.0000e+00 - prc: 0.0805
Epo

In [375]:
score_tfidf(m_tfidf2_final, test_tfidf[1], text_test_pairs[1][1])

{'accuracy': 0.9333876553269504,
 'recall': 0.0847953216374269,
 'precision': 0.6744186046511628,
 'auroc': 0.8575853087942102,
 'average precision': 0.3769995967523743,
 'f1': 0.15064935064935064}

Finally, we run the model on the 48 hour data.

In [83]:
m_tfidf3=model_tfidf(2)
m_tfidf3.fit(X_train_tfidf[2], y_train_tfidf[2], epochs=35, batch_size=30, verbose=1, validation_data=(X_val_tfidf[2], y_val_tfidf[2]))
#30+5

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_71 (Dense)             (None, 32)                3104064   
_________________________________________________________________
dense_72 (Dense)             (None, 1)                 33        
Total params: 3,104,097
Trainable params: 3,104,097
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/35
308/308 [==============================] - 21s 55ms/step - loss: 0.6003 - auc_18: 0.4923 - recall_18: 0.0208 - precision_18: 0.2500 - prc: 0.0892 - val_loss: 0.4088 - val_auc_18: 0.4387 - val_recall_18: 0.0000e+00 - val_precision_18: 0.0000e+00 - val_prc: 0.0872
Epoch 2/35
308/308 [==============================] - 14s 44ms/step - loss: 0.3782 - auc_18: 0.4762 - recall_18: 0.0000e+00 - precision_18: 0.0000e+00 - prc: 0.1078 - val_loss: 0.3436 - val_auc_18: 0.4507 - val_recall_18: 0.000

In [84]:
m_tfidf3_final=model_tfidf(2)
m_tfidf3_final.fit(train_tfidf[2], text_train_pairs[2][1], epochs=35, batch_size=30, verbose=1)
score_tfidf(m_tfidf3_final, test_tfidf[2], text_test_pairs[2][1])

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_73 (Dense)             (None, 32)                3104064   
_________________________________________________________________
dense_74 (Dense)             (None, 1)                 33        
Total params: 3,104,097
Trainable params: 3,104,097
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/35
385/385 [==============================] - 18s 38ms/step - loss: 0.5749 - auc_19: 0.4989 - recall_19: 0.0055 - precision_19: 0.1252 - prc: 0.0962
Epoch 2/35
385/385 [==============================] - 15s 40ms/step - loss: 0.3494 - auc_19: 0.4537 - recall_19: 0.0000e+00 - precision_19: 0.0000e+00 - prc: 0.0865
Epoch 3/35
385/385 [==============================] - 14s 37ms/step - loss: 0.3300 - auc_19: 0.4981 - recall_19: 0.0000e+00 - precision_19: 0.0000e+00 - prc: 0.0994
Epoch 4/35
385

{'accuracy': 0.9150349650349651,
 'recall': 0.025,
 'precision': 0.4,
 'auroc': 0.8113263358778626,
 'average precision': 0.30221183579796795,
 'f1': 0.04705882352941177}

### Hybrid models using TF-IDF

In this section, we combine the previous two sets of neural network models to constructure a two input neural network model that runs on both the structured and unstructured data.

In [14]:
import scipy
train_num_temp=scipy.sparse.csr_matrix(X_train_split_num[0].values)
val_num_temp=scipy.sparse.csr_matrix(X_val_split_num[0].values)

In [25]:
tfidf_input=Input(shape=(X_train_split_tfidf[0].shape[1], ), dtype='float32', name='tfidf')
tfidf_output1=Dense(32,  activation='relu', 
                      kernel_regularizer=tf.keras.regularizers.l2(l2=0.0001))(tfidf_input)


obs_input=Input(shape=(X_train_split_num[0].shape[1],), dtype='float32', name='obs')
encoded=Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.005))(obs_input)
encoded2=Dropout(0.5)(encoded)
encoded3=Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.005))(encoded2)
encoded4=Dropout(0.3)(encoded3)
concatenated=layers.concatenate([encoded4,tfidf_output1], axis=-1)
encoded5=Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.005))(concatenated)


output=Dense(1, activation='sigmoid')(encoded5)
modelB_tfidf=Model([tfidf_input, obs_input], output)

opt = keras.optimizers.Adam(learning_rate=0.0001)
modelB_tfidf.compile(loss='binary_crossentropy', optimizer=opt, metrics=[keras.metrics.AUC(), keras.metrics.Recall(), keras.metrics.Precision(), keras.metrics.AUC(name='prc', curve='PR')])
print(modelB_tfidf.summary())
#modelB.save_weights('modelB.h5')

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
obs (InputLayer)                [(None, 1405)]       0                                            
__________________________________________________________________________________________________
dense_14 (Dense)                (None, 512)          719872      obs[0][0]                        
__________________________________________________________________________________________________
dropout_6 (Dropout)             (None, 512)          0           dense_14[0][0]                   
__________________________________________________________________________________________________
dense_15 (Dense)                (None, 128)          65664       dropout_6[0][0]                  
____________________________________________________________________________________________

In [18]:
for i in [0, 1, 2]:
    X_train_split_tfidf[i]=pd.DataFrame.sparse.from_spmatrix(X_train_split_tfidf[i])
    X_val_split_tfidf[i]=pd.DataFrame.sparse.from_spmatrix(X_val_split_tfidf[i])

We train the model, scoring it on the validation set to tune hyperparameters.

In [19]:
modelB_tfidf.fit([X_train_split_tfidf[0], train_num_temp], y_train_split[0], verbose=1,
                validation_data=([X_val_split_tfidf[0], val_num_temp], y_val_split[0]),
                   epochs=30, batch_size=30)

Epoch 1/30
708/708 [==============================] - 46s 62ms/step - loss: 4.3944 - auc_2: 0.7003 - recall_2: 0.0909 - precision_2: 0.3996 - prc: 0.1914 - val_loss: 2.6838 - val_auc_2: 0.8636 - val_recall_2: 0.2478 - val_precision_2: 0.5570 - val_prc: 0.3918
Epoch 2/30
708/708 [==============================] - 43s 60ms/step - loss: 2.3623 - auc_2: 0.8640 - recall_2: 0.1367 - precision_2: 0.5458 - prc: 0.3651 - val_loss: 1.6002 - val_auc_2: 0.8735 - val_recall_2: 0.2418 - val_precision_2: 0.5912 - val_prc: 0.4218
Epoch 3/30
708/708 [==============================] - 44s 62ms/step - loss: 1.4087 - auc_2: 0.8962 - recall_2: 0.1637 - precision_2: 0.6664 - prc: 0.4439 - val_loss: 0.9887 - val_auc_2: 0.8823 - val_recall_2: 0.2269 - val_precision_2: 0.6230 - val_prc: 0.4410
Epoch 4/30
708/708 [==============================] - 47s 67ms/step - loss: 0.8781 - auc_2: 0.9050 - recall_2: 0.1797 - precision_2: 0.6999 - prc: 0.4877 - val_loss: 0.6444 - val_auc_2: 0.8874 - val_recall_2: 0.2060 - va

KeyboardInterrupt: 

Finally, we train it on the full training set and score the model on the test set.

In [26]:
tfidf_input=Input(shape=(train_tfidf[0].shape[1], ), dtype='float32', name='tfidf')
#tfidf_output1=model_tfidf.get_layer('dense1')(tfidf_input)
tfidf_output1=Dense(32,  activation='relu', 
                      kernel_regularizer=tf.keras.regularizers.l2(l2=0.0001))(tfidf_input)


obs_input=Input(shape=(X_train_split_num[0].shape[1],), dtype='float32', name='obs')
encoded=Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.005))(obs_input)
encoded2=Dropout(0.5)(encoded)
encoded3=Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.005))(encoded2)
encoded4=Dropout(0.3)(encoded3)
concatenated=layers.concatenate([encoded4,tfidf_output1], axis=-1)
encoded5=Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.005))(concatenated)


output=Dense(1, activation='sigmoid')(encoded5)
modelB_tfidf=Model([tfidf_input, obs_input], output)

opt = keras.optimizers.Adam(learning_rate=0.0001)
modelB_tfidf.compile(loss='binary_crossentropy', optimizer=opt, metrics=[keras.metrics.AUC(), keras.metrics.Recall(), keras.metrics.Precision(), keras.metrics.AUC(name='prc', curve='PR')])
print(modelB_tfidf.summary())
#modelB.save_weights('modelB.h5')

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
obs (InputLayer)                [(None, 1405)]       0                                            
__________________________________________________________________________________________________
dense_19 (Dense)                (None, 512)          719872      obs[0][0]                        
__________________________________________________________________________________________________
dropout_8 (Dropout)             (None, 512)          0           dense_19[0][0]                   
__________________________________________________________________________________________________
dense_20 (Dense)                (None, 128)          65664       dropout_8[0][0]                  
____________________________________________________________________________________________

In [27]:
modelB_tfidf.fit([train_tfidf[0], X_scaled[0]], text_train_pairs[0][1], epochs=20, batch_size=30)

Epoch 1/20
885/885 [==============================] - 48s 52ms/step - loss: 4.3199 - auc_5: 0.7134 - recall_5: 0.0744 - precision_5: 0.2541 - prc: 0.1624
Epoch 2/20
885/885 [==============================] - 43s 49ms/step - loss: 2.0864 - auc_5: 0.8512 - recall_5: 0.1522 - precision_5: 0.6006 - prc: 0.3807
Epoch 3/20
885/885 [==============================] - 46s 52ms/step - loss: 1.1337 - auc_5: 0.8798 - recall_5: 0.1762 - precision_5: 0.6439 - prc: 0.4376
Epoch 4/20
885/885 [==============================] - 42s 47ms/step - loss: 0.6672 - auc_5: 0.8921 - recall_5: 0.1654 - precision_5: 0.6431 - prc: 0.4543
Epoch 5/20
885/885 [==============================] - 44s 49ms/step - loss: 0.4372 - auc_5: 0.8920 - recall_5: 0.1695 - precision_5: 0.6885 - prc: 0.4622
Epoch 6/20
885/885 [==============================] - 42s 47ms/step - loss: 0.3145 - auc_5: 0.8997 - recall_5: 0.1795 - precision_5: 0.7139 - prc: 0.4788
Epoch 7/20
885/885 [==============================] - 43s 48ms/step - loss: 

In [37]:
def score_nn_tfidf(trained_model, test_tfidfs, test_scaled, y_test):
    y_prob=trained_model.predict([test_tfidfs, test_scaled])
    y_pred=y_prob.copy()
    y_pred[y_prob<0.5]=0
    y_pred[y_prob>=0.5]=1
    cc=accuracy_score(y_test, y_pred)
    rec=recall_score(y_test, y_pred)
    prec=precision_score(y_test, y_pred)
    auroc=roc_auc_score(y_test, y_prob)
    ap=average_precision_score(y_test, y_prob)
    f1=f1_score(y_test, y_pred)
    score={'accuracy':acc, 'recall':rec, 'precision':prec, 'auroc':auroc, 'average precision':ap, 'f1':f1}
    return score

In [38]:
score_nn_tfidf(modelB_tfidf, test_tfidf[0], X_scaled_test[0], text_test_pairs[0][1])

{'accuracy': 0.9478365747022464,
 'recall': 0.25059665871121717,
 'precision': 0.7664233576642335,
 'auroc': 0.9269503077583683,
 'average precision': 0.5764181925079449,
 'f1': 0.37769784172661863}

Next, we do the same thing on the 24 hour data.

In [41]:
train_num_temp=scipy.sparse.csr_matrix(X_train_split_num[1].values)
val_num_temp=scipy.sparse.csr_matrix(X_val_split_num[1].values)

In [44]:
tfidf_input=Input(shape=(X_train_split_tfidf[1].shape[1], ), dtype='float32', name='tfidf')
tfidf_output1=Dense(32,  activation='relu', 
                      kernel_regularizer=tf.keras.regularizers.l2(l2=0.0001))(tfidf_input)


obs_input=Input(shape=(X_train_split_num[1].shape[1],), dtype='float32', name='obs')
encoded=Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.005))(obs_input)
encoded2=Dropout(0.5)(encoded)
encoded3=Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.005))(encoded2)
encoded4=Dropout(0.3)(encoded3)
concatenated=layers.concatenate([encoded4,tfidf_output1], axis=-1)
encoded5=Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.005))(concatenated)


output=Dense(1, activation='sigmoid')(encoded5)
modelB_tfidf2=Model([tfidf_input, obs_input], output)

opt = keras.optimizers.Adam(learning_rate=0.0001)
modelB_tfidf2.compile(loss='binary_crossentropy', optimizer=opt, metrics=[keras.metrics.AUC(), keras.metrics.Recall(), keras.metrics.Precision(), keras.metrics.AUC(name='prc', curve='PR')])
print(modelB_tfidf2.summary())

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
obs (InputLayer)                [(None, 2653)]       0                                            
__________________________________________________________________________________________________
dense_30 (Dense)                (None, 512)          1358848     obs[0][0]                        
__________________________________________________________________________________________________
dropout_12 (Dropout)            (None, 512)          0           dense_30[0][0]                   
__________________________________________________________________________________________________
dense_31 (Dense)                (None, 128)          65664       dropout_12[0][0]                 
____________________________________________________________________________________________

In [45]:
modelB_tfidf2.fit([X_train_split_tfidf[1], train_num_temp], y_train_split[1], verbose=1,
                validation_data=([X_val_split_tfidf[1], val_num_temp], y_val_split[1]),
                   epochs=30, batch_size=30)

Epoch 1/30
524/524 [==============================] - 52s 91ms/step - loss: 4.9353 - auc_8: 0.6851 - recall_8: 0.0357 - precision_8: 0.3740 - prc: 0.1866 - val_loss: 3.1923 - val_auc_8: 0.8729 - val_recall_8: 0.2222 - val_precision_8: 0.5849 - val_prc: 0.4209
Epoch 2/30
524/524 [==============================] - 44s 83ms/step - loss: 2.8736 - auc_8: 0.8429 - recall_8: 0.1364 - precision_8: 0.6296 - prc: 0.3779 - val_loss: 2.0306 - val_auc_8: 0.8826 - val_recall_8: 0.2832 - val_precision_8: 0.5852 - val_prc: 0.4563
Epoch 3/30
524/524 [==============================] - 42s 81ms/step - loss: 1.8438 - auc_8: 0.8637 - recall_8: 0.1967 - precision_8: 0.6720 - prc: 0.4291 - val_loss: 1.3409 - val_auc_8: 0.8894 - val_recall_8: 0.3011 - val_precision_8: 0.5455 - val_prc: 0.4532
Epoch 4/30
524/524 [==============================] - 46s 89ms/step - loss: 1.2181 - auc_8: 0.8813 - recall_8: 0.1928 - precision_8: 0.6636 - prc: 0.4297 - val_loss: 0.9214 - val_auc_8: 0.8928 - val_recall_8: 0.2975 - va

In [48]:
tfidf_input=Input(shape=(train_tfidf[1].shape[1], ), dtype='float32', name='tfidf')
#tfidf_output1=model_tfidf.get_layer('dense1')(tfidf_input)
tfidf_output1=Dense(32,  activation='relu', 
                      kernel_regularizer=tf.keras.regularizers.l2(l2=0.0001))(tfidf_input)


obs_input=Input(shape=(X_scaled[1].shape[1],), dtype='float32', name='obs')
encoded=Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.005))(obs_input)
encoded2=Dropout(0.5)(encoded)
encoded3=Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.005))(encoded2)
encoded4=Dropout(0.3)(encoded3)
concatenated=layers.concatenate([encoded4,tfidf_output1], axis=-1)
encoded5=Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.005))(concatenated)


output=Dense(1, activation='sigmoid')(encoded5)
modelB_tfidf2=Model([tfidf_input, obs_input], output)

opt = keras.optimizers.Adam(learning_rate=0.0001)
modelB_tfidf2.compile(loss='binary_crossentropy', optimizer=opt, metrics=[keras.metrics.AUC(), keras.metrics.Recall(), keras.metrics.Precision(), keras.metrics.AUC(name='prc', curve='PR')])
print(modelB_tfidf2.summary())

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
obs (InputLayer)                [(None, 2653)]       0                                            
__________________________________________________________________________________________________
dense_40 (Dense)                (None, 512)          1358848     obs[0][0]                        
__________________________________________________________________________________________________
dropout_16 (Dropout)            (None, 512)          0           dense_40[0][0]                   
__________________________________________________________________________________________________
dense_41 (Dense)                (None, 128)          65664       dropout_16[0][0]                 
____________________________________________________________________________________________

In [52]:
modelB_tfidf2.fit([train_tfidf[1], X_scaled[1]], text_train_pairs[1][1], epochs=19, batch_size=30)

Epoch 1/19
655/655 [==============================] - 35s 50ms/step - loss: 4.9546 - auc_10: 0.7206 - recall_10: 0.1099 - precision_10: 0.3213 - prc: 0.1974
Epoch 2/19
655/655 [==============================] - 32s 49ms/step - loss: 2.7953 - auc_10: 0.8476 - recall_10: 0.1532 - precision_10: 0.5419 - prc: 0.3718
Epoch 3/19
655/655 [==============================] - 30s 46ms/step - loss: 1.7120 - auc_10: 0.8690 - recall_10: 0.1781 - precision_10: 0.6269 - prc: 0.4188
Epoch 4/19
655/655 [==============================] - 31s 48ms/step - loss: 1.0883 - auc_10: 0.8813 - recall_10: 0.2013 - precision_10: 0.6396 - prc: 0.4625
Epoch 5/19
655/655 [==============================] - 31s 47ms/step - loss: 0.7354 - auc_10: 0.8907 - recall_10: 0.2155 - precision_10: 0.6582 - prc: 0.4681
Epoch 6/19
655/655 [==============================] - 31s 48ms/step - loss: 0.5175 - auc_10: 0.8979 - recall_10: 0.2356 - precision_10: 0.6942 - prc: 0.5053
Epoch 7/19
655/655 [==============================] - 31s 

In [53]:
score_nn_tfidf(modelB_tfidf2, test_tfidf[1], X_scaled_test[1], text_test_pairs[1][1])

{'accuracy': 0.9478365747022464,
 'recall': 0.2953216374269006,
 'precision': 0.7013888888888888,
 'auroc': 0.9170415272543815,
 'average precision': 0.5549321420841455,
 'f1': 0.4156378600823046}

Finally, we repeat the process for the 48 hour data.

In [65]:
train_num_temp=scipy.sparse.csr_matrix(X_train_split_num[2].values)
val_num_temp=scipy.sparse.csr_matrix(X_val_split_num[2].values)

In [67]:
tfidf_input=Input(shape=(X_train_split_tfidf[2].shape[1], ), dtype='float32', name='tfidf')
tfidf_output1=Dense(32,  activation='relu', 
                      kernel_regularizer=tf.keras.regularizers.l2(l2=0.0001))(tfidf_input)


obs_input=Input(shape=(X_train_split_num[2].shape[1],), dtype='float32', name='obs')
encoded=Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.005))(obs_input)
encoded2=Dropout(0.5)(encoded)
encoded3=Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.005))(encoded2)
encoded4=Dropout(0.3)(encoded3)
concatenated=layers.concatenate([encoded4,tfidf_output1], axis=-1)
encoded5=Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.005))(concatenated)


output=Dense(1, activation='sigmoid')(encoded5)
modelB_tfidf3=Model([tfidf_input, obs_input], output)

opt = keras.optimizers.Adam(learning_rate=0.0001)
modelB_tfidf3.compile(loss='binary_crossentropy', optimizer=opt, metrics=[keras.metrics.AUC(), keras.metrics.Recall(), keras.metrics.Precision(), keras.metrics.AUC(name='prc', curve='PR')])
print(modelB_tfidf3.summary())

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
obs (InputLayer)                [(None, 5149)]       0                                            
__________________________________________________________________________________________________
dense_56 (Dense)                (None, 512)          2636800     obs[0][0]                        
__________________________________________________________________________________________________
dropout_22 (Dropout)            (None, 512)          0           dense_56[0][0]                   
__________________________________________________________________________________________________
dense_57 (Dense)                (None, 128)          65664       dropout_22[0][0]                 
___________________________________________________________________________________________

In [70]:
modelB_tfidf3.fit([X_train_split_tfidf[2], train_num_temp], y_train_split[2], verbose=1,
                validation_data=([X_val_split_tfidf[2], val_num_temp], y_val_split[2]),
                   epochs=36, batch_size=30)

Epoch 1/5
308/308 [==============================] - 36s 118ms/step - loss: 0.2235 - auc_13: 0.9715 - recall_13: 0.6243 - precision_13: 0.8868 - prc: 0.8468 - val_loss: 0.3113 - val_auc_13: 0.8812 - val_recall_13: 0.3190 - val_precision_13: 0.6700 - val_prc: 0.5485
Epoch 2/5
308/308 [==============================] - 36s 116ms/step - loss: 0.2158 - auc_13: 0.9769 - recall_13: 0.6243 - precision_13: 0.9178 - prc: 0.8672 - val_loss: 0.3299 - val_auc_13: 0.8660 - val_recall_13: 0.3381 - val_precision_13: 0.6283 - val_prc: 0.4991
Epoch 3/5
308/308 [==============================] - 36s 116ms/step - loss: 0.2104 - auc_13: 0.9785 - recall_13: 0.6314 - precision_13: 0.9046 - prc: 0.8744 - val_loss: 0.3130 - val_auc_13: 0.8799 - val_recall_13: 0.3048 - val_precision_13: 0.7111 - val_prc: 0.5325
Epoch 4/5
308/308 [==============================] - 37s 119ms/step - loss: 0.2069 - auc_13: 0.9788 - recall_13: 0.6576 - precision_13: 0.9036 - prc: 0.8747 - val_loss: 0.3131 - val_auc_13: 0.8797 - val

In [71]:
tfidf_input=Input(shape=(train_tfidf[2].shape[1], ), dtype='float32', name='tfidf')
#tfidf_output1=model_tfidf.get_layer('dense1')(tfidf_input)
tfidf_output1=Dense(32,  activation='relu', 
                      kernel_regularizer=tf.keras.regularizers.l2(l2=0.0001))(tfidf_input)


obs_input=Input(shape=(X_scaled[2].shape[1],), dtype='float32', name='obs')
encoded=Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.005))(obs_input)
encoded2=Dropout(0.5)(encoded)
encoded3=Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.005))(encoded2)
encoded4=Dropout(0.3)(encoded3)
concatenated=layers.concatenate([encoded4,tfidf_output1], axis=-1)
encoded5=Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.005))(concatenated)


output=Dense(1, activation='sigmoid')(encoded5)
modelB_tfidf3=Model([tfidf_input, obs_input], output)

opt = keras.optimizers.Adam(learning_rate=0.0001)
modelB_tfidf3.compile(loss='binary_crossentropy', optimizer=opt, metrics=[keras.metrics.AUC(), keras.metrics.Recall(), keras.metrics.Precision(), keras.metrics.AUC(name='prc', curve='PR')])
print(modelB_tfidf3.summary())

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
obs (InputLayer)                [(None, 5149)]       0                                            
__________________________________________________________________________________________________
dense_61 (Dense)                (None, 512)          2636800     obs[0][0]                        
__________________________________________________________________________________________________
dropout_24 (Dropout)            (None, 512)          0           dense_61[0][0]                   
__________________________________________________________________________________________________
dense_62 (Dense)                (None, 128)          65664       dropout_24[0][0]                 
___________________________________________________________________________________________

In [72]:
modelB_tfidf3.fit([train_tfidf[2], X_scaled[2]], text_train_pairs[2][1], epochs=36, batch_size=30)

Epoch 1/36
385/385 [==============================] - 27s 61ms/step - loss: 5.7718 - auc_14: 0.6847 - recall_14: 0.1893 - precision_14: 0.2202 - prc: 0.1747
Epoch 2/36
385/385 [==============================] - 23s 61ms/step - loss: 3.9737 - auc_14: 0.8237 - recall_14: 0.1802 - precision_14: 0.5721 - prc: 0.3987
Epoch 3/36
385/385 [==============================] - 21s 54ms/step - loss: 2.8611 - auc_14: 0.8542 - recall_14: 0.2192 - precision_14: 0.6594 - prc: 0.4539
Epoch 4/36
385/385 [==============================] - 21s 54ms/step - loss: 2.0907 - auc_14: 0.8662 - recall_14: 0.2696 - precision_14: 0.6729 - prc: 0.4818
Epoch 5/36
385/385 [==============================] - 25s 64ms/step - loss: 1.5399 - auc_14: 0.8870 - recall_14: 0.3052 - precision_14: 0.7415 - prc: 0.5532
Epoch 6/36
385/385 [==============================] - 24s 62ms/step - loss: 1.2018 - auc_14: 0.8775 - recall_14: 0.2628 - precision_14: 0.6106 - prc: 0.5104
Epoch 7/36
385/385 [==============================] - 21s 

In [73]:
score_nn_tfidf(modelB_tfidf3, test_tfidf[2], X_scaled_test[2], text_test_pairs[2][1])

{'accuracy': 0.9478365747022464,
 'recall': 0.375,
 'precision': 0.6870229007633588,
 'auroc': 0.8890028625954197,
 'average precision': 0.5797896539159196,
 'f1': 0.48517520215633425}

## Bibliography

<div class="cite2c-biblio"></div>